Toma un archivo csv de ADNI y genera un archivo para train, otro para validation y otro para test.

ESTE ES DISTINTO A PreprocesarCSVADNI YA QUE CREA EL SET 80/10/10 USANDO LOS SUBJECT_ID EN VEZ DE IMAGE ID. LA IDEA ES NO REPETIR PACIENTES EN DISTINTOS GRUPOS.

Además agrega algunas columnas, entre ellas DX_last que es el label de la "última" visita con label

Genera 3 sets de sets: uno con DX_last como ground truth label, otro con DX_most_severe y otro con DX_visit.

In [5]:
import pandas as pd
import os
from os import path
import math
import numpy as np
import datetime
import random

In [6]:
trainProportion = 0.8
valProportion = 0.1
testProportion = 0.1
inputFile = 'ADNI-Full-PostProc/ADNI-FULL-PostProc_3_01_2023_UniformResolution.csv'
adniMergeCSV = '/Users/hugom/PET-IA/Data/ADNIMERGE.csv'
petMetaCSV = '/Users/hugom/PET-IA/Data/PET_META_LIST.csv'
csvDXSUMFile = '/Users/hugom/PET-IA/Data/DXSUM_PDXCONV_ADNIALL.csv'
outputFolder = '/Users/hugom/PET-IA/Sets'
outputFilesPrefix = 'Muestra3700_80_10_10'
randomState = 2234289712
minNumberOfStudies = 0 # Exclude patients with less than this number of studies
# prospectively collected 2109 18F-FDG PET imaging studies
# from 1002 patients, most patients with multiple scans, with
# dates ranging from May 2005 to January 2017,
fromDate = datetime.date(2000, 1, 1) # datetime.date(2005, 5, 1)
toDate = datetime.date(3000, 1, 1) # datetime.date(2017, 1, 31)
dxClasses = ['CN', 'AD', 'MCI'] # Permite excluír categorías
if len(dxClasses) < 3:
    outputFilesPrefix += '_' + ','.join(dxClasses)

random.seed(randomState)

In [7]:
dxLastTrainOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxlast_train.csv")
dxLastTrainMiniOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxlast_train_mini.csv")
dxLastValOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxlast_val.csv")
dxLastTestOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxlast_test.csv")

dxMostSevereTrainOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxmost_severe_train.csv")
dxMostSevereValOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxmost_severe_val.csv")
dxMostSevereTestOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxmost_severe_test.csv")

dxVisitTrainOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxvisit_train.csv")
dxVisitValOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxvisit_val.csv")
dxVisitTestOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxvisit_test.csv")

dxVis953TrainOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxvisit953_train.csv")
dxVis953ValOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxvisit953_val.csv")
dxVis953TestOutputFile = path.join(outputFolder, f"{outputFilesPrefix}_dxvisit953_test.csv")

In [8]:
csv = pd.read_csv(inputFile)
csv

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I1557047,941_S_7041,MCI,F,73,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,3/09/2022,DCM,11/29/2022
1,I1480671,941_S_6962,AD,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,8/10/2021,DCM,1/09/2023
2,I1301874,941_S_6854,AD,M,86,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2/24/2020,DCM,12/02/2022
3,I1239316,941_S_6803,MCI,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,10/07/2019,DCM,11/28/2022
4,I1005488,941_S_6345,MCI,M,79,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,5/30/2018,DCM,11/29/2022
...,...,...,...,...,...,...,...,...,...,...,...,...
3757,I187801,002_S_0685,CN,F,94,m48,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,7/27/2010,DCM,11/28/2022
3758,I323915,002_S_0685,CN,F,96,v11,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,8/02/2012,DCM,12/02/2022
3759,I384689,002_S_0413,CN,F,83,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,5/14/2013,DCM,11/28/2022
3760,I241476,002_S_0413,CN,F,82,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,6/17/2011,DCM,11/29/2022


In [9]:
len(csv['Image Data ID'].unique())

3762

In [10]:
csv['Subject'].describe()

count           3762
unique          1687
top       003_S_1122
freq              10
Name: Subject, dtype: object

In [11]:
csv['Image Data ID'].describe()

count         3762
unique        3762
top       I1557047
freq             1
Name: Image Data ID, dtype: object

In [12]:
csv['Visit'].unique()

array(['bl', 'v03', 'init', 'v21', 'm06', 'm12', 'm24', 'm18', 'm36',
       'v06', 'm48', 'v11', 'm60', 'v31'], dtype=object)

In [13]:
# Remove all studies that don't match the dates from Yiming Ding
csv['Acq Date'] = pd.to_datetime(csv['Acq Date']).dt.date
csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3762 entries, 0 to 3761
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Image Data ID  3762 non-null   object
 1   Subject        3762 non-null   object
 2   Group          3762 non-null   object
 3   Sex            3762 non-null   object
 4   Age            3762 non-null   int64 
 5   Visit          3762 non-null   object
 6   Modality       3762 non-null   object
 7   Description    3762 non-null   object
 8   Type           3762 non-null   object
 9   Acq Date       3762 non-null   object
 10  Format         3762 non-null   object
 11  Downloaded     3762 non-null   object
dtypes: int64(1), object(11)
memory usage: 352.8+ KB


In [14]:
csv

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I1557047,941_S_7041,MCI,F,73,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2022-03-09,DCM,11/29/2022
1,I1480671,941_S_6962,AD,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2021-08-10,DCM,1/09/2023
2,I1301874,941_S_6854,AD,M,86,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2020-02-24,DCM,12/02/2022
3,I1239316,941_S_6803,MCI,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2019-10-07,DCM,11/28/2022
4,I1005488,941_S_6345,MCI,M,79,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2018-05-30,DCM,11/29/2022
...,...,...,...,...,...,...,...,...,...,...,...,...
3757,I187801,002_S_0685,CN,F,94,m48,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2010-07-27,DCM,11/28/2022
3758,I323915,002_S_0685,CN,F,96,v11,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2012-08-02,DCM,12/02/2022
3759,I384689,002_S_0413,CN,F,83,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2013-05-14,DCM,11/28/2022
3760,I241476,002_S_0413,CN,F,82,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2011-06-17,DCM,11/29/2022


In [15]:
csv = csv[csv['Acq Date'] >= fromDate]
csv = csv[csv['Acq Date'] <= toDate]
csv

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,I1557047,941_S_7041,MCI,F,73,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2022-03-09,DCM,11/29/2022
1,I1480671,941_S_6962,AD,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2021-08-10,DCM,1/09/2023
2,I1301874,941_S_6854,AD,M,86,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2020-02-24,DCM,12/02/2022
3,I1239316,941_S_6803,MCI,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2019-10-07,DCM,11/28/2022
4,I1005488,941_S_6345,MCI,M,79,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2018-05-30,DCM,11/29/2022
...,...,...,...,...,...,...,...,...,...,...,...,...
3757,I187801,002_S_0685,CN,F,94,m48,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2010-07-27,DCM,11/28/2022
3758,I323915,002_S_0685,CN,F,96,v11,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2012-08-02,DCM,12/02/2022
3759,I384689,002_S_0413,CN,F,83,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2013-05-14,DCM,11/28/2022
3760,I241476,002_S_0413,CN,F,82,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2011-06-17,DCM,11/29/2022


In [16]:
csv2 = csv.groupby(by='Subject').aggregate({'Image Data ID': pd.Series.count, 'Group': pd.Series.unique })

In [17]:
# Veamos cuántos tienen mas de un valor en la columna Group
csv3 = csv2[csv2['Group'].apply(lambda x: len(x) > 1)]
csv3

,Image Data ID,Group
Subject,,


Lo de arriba quiere decir que no hay ningún paciente cuyo grupo haya cambiado. Según ChatGPT:

The "Group" column in the ADNI dataset is based on the participant's diagnostic group at the time of their baseline assessment, which is determined by clinical evaluation, cognitive testing, and other diagnostic criteria.

Once a participant is assigned to a diagnostic group (CN, MCI, or AD) at baseline, their group status is considered fixed and does not change in subsequent visits or assessments. This is because the ADNI study aims to track the natural history of Alzheimer's disease and other forms of cognitive decline, and changes in diagnostic group status would complicate the interpretation of the data.

However, it's important to note that participants' cognitive and clinical status can change over time, and this is reflected in other variables in the ADNI dataset, such as cognitive test scores, biomarkers, and clinical assessments. So while the "Group" column remains constant, other variables can change and provide important information about disease progression and treatment efficacy.

In [18]:
csv2 = csv.groupby(by='Subject')
csv2['Group'].describe()

,count,unique,top,freq
Subject,,,,
002_S_0295,1,1,CN,1
002_S_0413,2,1,CN,2
002_S_0685,2,1,CN,2
002_S_0729,2,1,MCI,2
002_S_1155,3,1,MCI,3
...,...,...,...,...
941_S_6345,1,1,MCI,1
941_S_6803,1,1,MCI,1
941_S_6854,1,1,AD,1


In [19]:
csv2.describe()

Age                                                     
           count       mean       std   min    25%   50%    75%   max
Subject                                                              
002_S_0295   1.0  90.000000       NaN  90.0  90.00  90.0  90.00  90.0
002_S_0413   2.0  82.500000  0.707107  82.0  82.25  82.5  82.75  83.0
002_S_0685   2.0  95.000000  1.414214  94.0  94.50  95.0  95.50  96.0
002_S_0729   2.0  70.000000  1.414214  69.0  69.50  70.0  70.50  71.0
002_S_1155   3.0  64.666667  3.055050  62.0  63.00  64.0  66.00  68.0
...          ...        ...       ...   ...    ...   ...    ...   ...
941_S_6345   1.0  79.000000       NaN  79.0  79.00  79.0  79.00  79.0
941_S_6803   1.0  75.000000       NaN  75.0  75.00  75.0  75.00  75.0
941_S_6854   1.0  86.000000       NaN  86.0  86.00  86.0  86.00  86.0
941_S_6962   1.0  75.000000       NaN  75.0  75.00  75.0  75.00  75.0
941_S_7041   1.0  73.000000       NaN  73.0  73.00  73.0  73.00  73.0

[1687 rows x 8 columns]

In [20]:
print("Número promedio de imágenes por paciente: ")
csv2.count()['Image Data ID'].mean()

Número promedio de imágenes por paciente: 


2.229994072317724

In [21]:
print("Desvío estándar de imágenes por paciente: ")
csv2.count()['Image Data ID'].std()

Desvío estándar de imágenes por paciente: 


1.7987016373028673

In [22]:
csv3 = csv2.filter(lambda x: len(x) >= 2 )
csv3

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
13,I277273,941_S_4365,CN,M,80,v03,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2011-12-30,DCM,1/09/2023
14,I901121,941_S_4365,CN,M,86,init,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2017-08-31,DCM,12/02/2022
15,I422985,941_S_4365,CN,M,83,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2014-04-16,DCM,12/02/2022
16,I398649,941_S_4292,CN,M,73,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2013-10-09,DCM,12/02/2022
17,I278425,941_S_4292,CN,M,71,v03,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2012-01-13,DCM,1/09/2023
...,...,...,...,...,...,...,...,...,...,...,...,...
3756,I189823,002_S_0729,MCI,F,69,m48,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2010-07-29,DCM,12/02/2022
3757,I187801,002_S_0685,CN,F,94,m48,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2010-07-27,DCM,11/28/2022
3758,I323915,002_S_0685,CN,F,96,v11,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2012-08-02,DCM,12/02/2022
3759,I384689,002_S_0413,CN,F,83,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2013-05-14,DCM,11/28/2022


In [23]:
csvMerge = pd.read_csv(adniMergeCSV,  dtype = {'DX' : str})
csvMerge
# Faltan eliminar todos aquellos que son de ADNI3 (Y Ding no usa ADNI3)

csvMerge = csvMerge[csvMerge['DX'].notna() ]
csvMerge.loc[csvMerge['DX'] == 'Dementia', 'DX'] = 'AD'
# Eliminamos todo lo referente a ADNI3
csvMerge = csvMerge[csvMerge['COLPROT'] != 'ADNI3']
csvMerge = csvMerge[csvMerge['ORIGPROT'] != 'ADNI3']
csvMerge.groupby(by='PTID').count()

/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_59484/2894782626.py:1: DtypeWarning: Columns (19,20,21,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  csvMerge = pd.read_csv(adniMergeCSV,  dtype = {'DX' : str})


,RID,COLPROT,ORIGPROT,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,PTEDUCAT,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
PTID,,,,,,,,,,,,,,,,,,,,,
002_S_0295,8,8,8,8,8,8,8,8,8,8,...,8,0,0,0,0,8,8,8,8,8
002_S_0413,11,11,11,11,11,11,11,11,11,11,...,11,0,0,0,0,11,11,11,11,11
002_S_0559,5,5,5,5,5,5,5,5,5,5,...,5,0,0,0,0,5,5,5,5,5
002_S_0619,4,4,4,4,4,4,4,4,4,4,...,4,0,0,0,0,4,4,4,4,4
002_S_0685,8,8,8,8,8,8,8,8,8,8,...,8,0,0,0,0,8,8,8,8,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941_S_4377,3,3,3,3,3,3,3,3,3,3,...,3,3,0,3,0,3,3,3,3,3
941_S_4420,3,3,3,3,3,3,3,3,3,3,...,3,3,0,3,0,3,3,3,3,3
941_S_4764,5,5,5,5,5,5,5,5,5,5,...,5,5,0,5,0,5,5,5,5,5


In [24]:
merge = pd.merge(
    csv,
    csvMerge,
    left_on = 'Subject',
    right_on = 'PTID',
    how = 'inner',
    copy = True
)
merge
merge.groupby(by='Subject').count()

,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
Subject,,,,,,,,,,,,,,,,,,,,,
002_S_0295,8,8,8,8,8,8,8,8,8,8,...,8,0,0,0,0,8,8,8,8,8
002_S_0413,22,22,22,22,22,22,22,22,22,22,...,22,0,0,0,0,22,22,22,22,22
002_S_0685,16,16,16,16,16,16,16,16,16,16,...,16,0,0,0,0,16,16,16,16,16
002_S_0729,20,20,20,20,20,20,20,20,20,20,...,20,0,0,0,0,20,20,20,20,20
002_S_1155,36,36,36,36,36,36,36,36,36,36,...,0,0,0,0,0,36,36,36,36,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
941_S_4377,3,3,3,3,3,3,3,3,3,3,...,3,3,0,3,0,3,3,3,3,3
941_S_4420,3,3,3,3,3,3,3,3,3,3,...,3,3,0,3,0,3,3,3,3,3
941_S_4764,5,5,5,5,5,5,5,5,5,5,...,5,5,0,5,0,5,5,5,5,5


In [25]:
csvMerge = merge[['PTID', 'DX', 'VISCODE', 'EXAMDATE']]
csvMerge

,PTID,DX,VISCODE,EXAMDATE
0,941_S_5193,CN,m24,2015-06-11
1,941_S_5193,CN,m06,2014-02-21
2,941_S_5193,CN,bl,2013-06-11
3,941_S_5124,CN,bl,2013-04-22
4,941_S_5124,CN,m06,2013-10-24
...,...,...,...,...
21887,002_S_0295,CN,m24,2008-08-04
21888,002_S_0295,CN,m36,2009-05-07
21889,002_S_0295,CN,m48,2010-05-13
21890,002_S_0295,CN,m72,2012-05-10


Lo de arriba no se usa, no se eliminan los de ADNI3 porque finalmente no tenemos el criterio de selección de Y Ding

# DX_last

In [26]:
csvMerge = pd.read_csv(adniMergeCSV,  dtype = {'DX' : str})
csvMerge['IMAGEUID'].describe()

/var/folders/hy/rbnyh5md5yvc72ynxb0c2cz40000gn/T/ipykernel_59484/2463587978.py:1: DtypeWarning: Columns (19,20,21,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  csvMerge = pd.read_csv(adniMergeCSV,  dtype = {'DX' : str})


count    9.933000e+03
mean     4.131512e+05
std      3.836951e+05
min      3.186300e+04
25%      1.195060e+05
50%      2.993040e+05
75%      4.324260e+05
max      1.570686e+06
Name: IMAGEUID, dtype: float64

In [27]:
csvMerge['DX'].unique()

array(['CN', 'Dementia', 'MCI', nan], dtype=object)

In [28]:
len(csvMerge[csvMerge['DX'].isna()])

4945

In [29]:
# Eliminamos los que tengan DX = nan
csvMerge = csvMerge[csvMerge['DX'].notna() ]
len(csvMerge[csvMerge['DX'].isna()])

0

In [30]:
csvMerge['VISCODE'].unique()

array(['bl', 'm06', 'm12', 'm24', 'm18', 'm36', 'm48', 'm60', 'm72',
       'm108', 'm96', 'm84', 'm120', 'm132', 'm144', 'm156', 'm168',
       'm180', 'm42', 'm174', 'm162', 'm138', 'm150', 'm102', 'm90',
       'm114', 'm78', 'm126', 'm66', 'm54', 'm30', 'm186', 'm192', 'm198'],
      dtype=object)

In [31]:
len(csvMerge['VISCODE'].unique())

34

In [32]:
len(csvMerge['PTID'].unique()) ### ??? por qué hay 2406???

2406

In [33]:
# Renombramos Dementia como AD
csvMerge.loc[csvMerge['DX'] == 'Dementia', 'DX'] = 'AD'
csvMerge

,RID,COLPROT,ORIGPROT,PTID,SITE,VISCODE,EXAMDATE,DX_bl,AGE,PTGENDER,...,PTAU_bl,FDG_bl,PIB_bl,AV45_bl,FBB_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,ADNI1,ADNI1,011_S_0002,11,bl,2005-09-08,CN,74.3,Male,...,NaN,1.33615,NaN,NaN,NaN,0.000000,0.00000,0,0,2022-10-26 04:58:48.0
1,3,ADNI1,ADNI1,011_S_0003,11,bl,2005-09-12,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,0.000000,0.00000,0,0,2022-10-26 04:58:48.0
2,3,ADNI1,ADNI1,011_S_0003,11,m06,2006-03-13,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,0.498289,5.96721,6,6,2022-10-26 04:58:48.0
3,3,ADNI1,ADNI1,011_S_0003,11,m12,2006-09-12,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,0.999316,11.96720,12,12,2022-10-26 04:58:48.0
4,3,ADNI1,ADNI1,011_S_0003,11,m24,2007-09-12,AD,81.3,Male,...,22.83,1.10860,NaN,NaN,NaN,1.998630,23.93440,24,24,2022-10-26 04:58:48.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16311,6668,ADNI3,ADNI3,019_S_6668,19,m48,2023-03-31,LMCI,75.6,Female,...,NaN,1.13922,NaN,NaN,1.6525,4.145110,49.63930,48,48,2023-04-05 14:11:55.0
16317,6869,ADNI3,ADNI3,057_S_6869,57,m24,2023-03-07,AD,78.3,Male,...,NaN,NaN,NaN,NaN,1.4979,1.993160,23.86890,24,24,2023-04-05 14:11:56.0
16326,7117,ADNI3,ADNI3,082_S_7117,82,bl,2023-01-12,EMCI,68.9,Female,...,NaN,NaN,NaN,NaN,NaN,0.000000,0.00000,0,0,2023-03-21 04:58:36.0
16329,7021,ADNI3,ADNI3,035_S_7021,35,m12,2023-03-20,LMCI,67.4,Female,...,NaN,NaN,NaN,NaN,NaN,0.969199,11.60660,12,12,2023-04-05 14:11:56.0


In [34]:
csvMerge['DX']

0         CN
1         AD
2         AD
3         AD
4         AD
        ... 
16311     AD
16317     AD
16326    MCI
16329    MCI
16338    MCI
Name: DX, Length: 11396, dtype: object

In [35]:
csvMerge['EXAMDATE'] = pd.to_datetime(csvMerge['EXAMDATE'])

In [36]:
# "agrupamos por patient"
csvMergePerPatient = csvMerge.groupby(by='PTID').agg({'DX': list, 'VISCODE': list, 'EXAMDATE': [np.min, np.max]})

In [37]:
# csvMergePerPatient['PTID'] = csvMergePerPatient.index
csvMergePerPatient = csvMergePerPatient.reset_index()
csvMergePerPatient

PTID                                                 DX  \
                                                               list   
0     002_S_0295                   [CN, CN, CN, CN, CN, CN, CN, CN]   
1     002_S_0413  [CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, C...   
2     002_S_0559                               [CN, CN, CN, CN, CN]   
3     002_S_0619                                   [AD, AD, AD, AD]   
4     002_S_0685                   [CN, CN, CN, CN, CN, CN, CN, CN]   
...          ...                                                ...   
2401  941_S_7041                                              [MCI]   
2402  941_S_7046                                               [CN]   
2403  941_S_7051                                               [CN]   
2404  941_S_7074                                               [CN]   
2405  941_S_7085                                              [MCI]   

                                                VISCODE   EXAMDATE             
                                                   list       amin       amax  
0               [bl, m06, m12, m24, m36, m48, m72, m60] 2006-05-09 2012-05-10  
1     [bl, m06, m12, m24, m36, m48, m108, m96, m84, ... 2006-05-08 2019-08-22  
2                              [bl, m24, m36, m06, m12] 2006-06-20 2009-07-02  
3                                   [bl, m06, m12, m24] 2006-06-20 2008-08-07  
4               [bl, m06, m12, m24, m36, m48, m60, m72] 2006-07-20 2012-07-27  
...                                                 ...        ...        ...  
2401                                               [bl] 2022-03-01 2022-03-01  
2402                                               [bl] 2022-04-06 2022-04-06  
2403                                               [bl] 2022-03-30 2022-03-30  
2404                                               [bl] 2022-05-02 2022-05-02  
2405                                               [bl] 2022-07-11 2022-07-11  

[2406 rows x 5 columns]

In [38]:
# calculate the length in months between the first and last EXAMDATE for each patient
csvMergePerPatient['EXAMLENGTH'] = (csvMergePerPatient['EXAMDATE']['amax'] - csvMergePerPatient['EXAMDATE']['amin'])/np.timedelta64(1, 'M')

# print the resulting DataFrame
csvMergePerPatient

PTID                                                 DX  \
                                                               list   
0     002_S_0295                   [CN, CN, CN, CN, CN, CN, CN, CN]   
1     002_S_0413  [CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, C...   
2     002_S_0559                               [CN, CN, CN, CN, CN]   
3     002_S_0619                                   [AD, AD, AD, AD]   
4     002_S_0685                   [CN, CN, CN, CN, CN, CN, CN, CN]   
...          ...                                                ...   
2401  941_S_7041                                              [MCI]   
2402  941_S_7046                                               [CN]   
2403  941_S_7051                                               [CN]   
2404  941_S_7074                                               [CN]   
2405  941_S_7085                                              [MCI]   

                                                VISCODE   EXAMDATE             \
                                                   list       amin       amax   
0               [bl, m06, m12, m24, m36, m48, m72, m60] 2006-05-09 2012-05-10   
1     [bl, m06, m12, m24, m36, m48, m108, m96, m84, ... 2006-05-08 2019-08-22   
2                              [bl, m24, m36, m06, m12] 2006-06-20 2009-07-02   
3                                   [bl, m06, m12, m24] 2006-06-20 2008-08-07   
4               [bl, m06, m12, m24, m36, m48, m60, m72] 2006-07-20 2012-07-27   
...                                                 ...        ...        ...   
2401                                               [bl] 2022-03-01 2022-03-01   
2402                                               [bl] 2022-04-06 2022-04-06   
2403                                               [bl] 2022-03-30 2022-03-30   
2404                                               [bl] 2022-05-02 2022-05-02   
2405                                               [bl] 2022-07-11 2022-07-11   

      EXAMLENGTH  
                  
0      72.050761  
1     159.477607  
2      36.403212  
3      25.593955  
4      72.247890  
...          ...  
2401    0.000000  
2402    0.000000  
2403    0.000000  
2404    0.000000  
2405    0.000000  

[2406 rows x 6 columns]

In [39]:
# Avoiding multi index to avoid pain
csvMergePerPatient.columns = ['PTID', 'DX', 'VISCODE', 'EXAMDATE_min', 'EXAMDATE_max', 'EXAMLENGTH']

In [40]:
csvMergePerPatient['EXAMLENGTH'].mean()

43.32103545389864

In [41]:
csvMergePerPatient['EXAMLENGTH'].std()

40.09794032221812

In [42]:
len(csvMergePerPatient[csvMergePerPatient['EXAMLENGTH'] == 0.0])

334

In [43]:
len(csvMergePerPatient)

2406

Si quisiéramos quedarnos con los que estuvieron al menos 2 años...

In [44]:
len(csvMergePerPatient[csvMergePerPatient['EXAMLENGTH'] >= 24.0 ])

1527

Es decir, nos quedaríamos con 1527 pacientes...

In [45]:
csvMergePerPatient['EXAMLENGTH'].describe()

count    2406.000000
mean       43.321035
std        40.097940
min         0.000000
25%        12.353436
50%        35.368283
75%        59.960163
max       196.800756
Name: EXAMLENGTH, dtype: float64

In [46]:
csvMergePerPatient['DX_last'] = csvMergePerPatient['DX'].apply(lambda xs: xs[-1])

In [47]:
csvMergePerPatient['DX_last']

0        CN
1        CN
2        CN
3        AD
4        CN
       ... 
2401    MCI
2402     CN
2403     CN
2404     CN
2405    MCI
Name: DX_last, Length: 2406, dtype: object

In [48]:
csvMergePerPatient['DX_last'].describe()

count     2406
unique       3
top        MCI
freq       874
Name: DX_last, dtype: object

In [49]:
csvMergePerPatient.groupby(by='DX_last').describe()

EXAMLENGTH                                                   \
             count       mean        std  min        25%        50%   
DX_last                                                               
AD           681.0  35.095819  33.517289  0.0  12.222017  24.148340   
CN           851.0  49.245031  43.953776  0.0  13.930471  37.914536   
MCI          874.0  43.961827  39.823616  0.0  12.427360  36.107518   

                                
               75%         max  
DX_last                         
AD       47.541017  191.806813  
CN       73.249964  196.800756  
MCI      60.075156  192.299637

In [50]:
# Devuelve el diagnóstico mas severo de una lista de diagnósticos
def dxGetMostSevere(xs):
    mostSevere = 'CN'
    for x in xs:
        if x == 'MCI' and mostSevere == 'CN':
            mostSevere = 'MCI'
        if x == 'AD':
            mostSevere = 'AD'
    return mostSevere

csvMergePerPatient['DX_most_severe'] = csvMergePerPatient['DX'].apply(dxGetMostSevere)

In [51]:
csvMergePerPatient['DX_most_severe']

0        CN
1        CN
2        CN
3        AD
4        CN
       ... 
2401    MCI
2402     CN
2403     CN
2404     CN
2405    MCI
Name: DX_most_severe, Length: 2406, dtype: object

In [52]:
csvMergePerPatient.groupby(by='DX_most_severe').describe()

EXAMLENGTH                                                   \
                    count       mean        std  min        25%        50%   
DX_most_severe                                                               
AD                  825.0  38.256665  33.953485  0.0  12.517711  24.411179   
CN                  746.0  45.240118  42.433177  0.0  11.778476  36.156800   
MCI                 835.0  46.610221  43.006870  0.0  12.353436  36.107518   

                                       
                      75%         max  
DX_most_severe                         
AD              48.986632  191.806813  
CN              61.487916  188.817019  
MCI             65.364792  196.800756

In [53]:
def dxHasRegression(xs):
    regressions = 0
    current = 'CN'
    for x in xs:
        if current == 'MCI':
            if x == 'CN':
                regressions += 1
                
        if current == 'AD':
            if x == 'MCI' or x == 'CN':
                regressions += 1
                
        current = x

    return regressions    
csvMergePerPatient['regressions'] = csvMergePerPatient['DX'].apply(dxHasRegression)

In [54]:
csvMergePerPatient['regressions'].describe()

count    2406.000000
mean        0.165004
std         0.418637
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         4.000000
Name: regressions, dtype: float64

In [55]:
len(csvMergePerPatient[csvMergePerPatient['regressions'] > 0])

356

In [56]:
len(csvMergePerPatient[csvMergePerPatient['regressions'] > 1])

38

In [57]:
len(csvMergePerPatient[csvMergePerPatient['regressions'] > 2])

2

In [58]:
len(csvMergePerPatient[csvMergePerPatient['regressions'] > 3])

1

In [59]:
len(csvMergePerPatient[csvMergePerPatient['regressions'] > 4])

0

In [60]:
len(csvMergePerPatient[csvMergePerPatient['DX_last'] != csvMergePerPatient['DX_most_severe']])

243

In [61]:
csvMergePerPatientDiff = csvMergePerPatient[csvMergePerPatient['DX_last'] != csvMergePerPatient['DX_most_severe']][['PTID', 'DX_last', 'DX_most_severe']]
csvMergePerPatientDiff

,PTID,DX_last,DX_most_severe
12,002_S_1070,MCI,AD
15,002_S_1268,MCI,AD
17,002_S_2010,CN,MCI
20,002_S_4171,MCI,AD
27,002_S_4262,CN,AD
...,...,...,...
2321,301_S_6297,CN,MCI
2322,301_S_6326,CN,MCI
2334,305_S_6810,MCI,AD
2337,305_S_6871,CN,MCI


In [62]:
csvMergePerPatientDiff.groupby(by=['DX_last', 'DX_most_severe']).describe()

PTID                        
                       count unique         top freq
DX_last DX_most_severe                              
CN      AD                 6      6  002_S_4262    1
        MCI               99     99  002_S_2010    1
MCI     AD               138    138  002_S_1070    1

In [63]:
csvMergePerPatient['DX_hashed'] = csvMergePerPatient['DX'].apply(lambda xs: ','.join(xs))

In [64]:
csvMergePerPatient['DX_hashed'].unique()

array(['CN,CN,CN,CN,CN,CN,CN,CN',
       'CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN', 'CN,CN,CN,CN,CN',
       'AD,AD,AD,AD', 'MCI,MCI,AD,AD,AD,AD,AD,AD,AD,AD',
       'MCI,MCI,MCI,MCI,MCI,MCI', 'AD,AD,AD', 'MCI,MCI,AD,AD,AD', 'AD,AD',
       'MCI,MCI,AD,AD,AD,MCI',
       'MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI',
       'CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,MCI,MCI,MCI',
       'MCI,MCI,MCI,MCI,MCI,MCI,MCI,AD,MCI',
       'CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN', 'MCI,CN,CN,CN,CN,CN',
       'MCI,MCI,MCI', 'MCI,MCI,CN,CN,CN,MCI,MCI,MCI', 'AD,MCI,MCI,MCI',
       'CN,CN,CN,CN,CN,CN,CN', 'CN,CN,CN,CN,CN,AD,AD',
       'MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI', 'MCI,MCI,MCI,MCI',
       'MCI,MCI', 'AD,MCI,MCI,MCI,CN,CN,CN', 'MCI,CN,CN,CN,CN',
       'MCI,MCI,MCI,MCI,MCI', 'MCI,MCI,MCI,MCI,MCI,MCI,MCI',
       'AD,AD,MCI,MCI,MCI,MCI', 'MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI,MCI',
       'MCI,AD,MCI,MCI,MCI,MCI', 'MCI,MCI,CN,CN,MCI,MCI,CN,CN,CN',
       'CN,CN,CN,CN,CN,CN', 'CN,CN,C

In [65]:
len(csvMergePerPatient['DX_hashed'].unique())

369

In [66]:
len(csv)

3762

In [67]:
csvMergePerPatient

,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
0,002_S_0295,"[CN, CN, CN, CN, CN, CN, CN, CN]","[bl, m06, m12, m24, m36, m48, m72, m60]",2006-05-09,2012-05-10,72.050761,CN,CN,0,"CN,CN,CN,CN,CN,CN,CN,CN"
1,002_S_0413,"[CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, C...","[bl, m06, m12, m24, m36, m48, m108, m96, m84, ...",2006-05-08,2019-08-22,159.477607,CN,CN,0,"CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN"
2,002_S_0559,"[CN, CN, CN, CN, CN]","[bl, m24, m36, m06, m12]",2006-06-20,2009-07-02,36.403212,CN,CN,0,"CN,CN,CN,CN,CN"
3,002_S_0619,"[AD, AD, AD, AD]","[bl, m06, m12, m24]",2006-06-20,2008-08-07,25.593955,AD,AD,0,"AD,AD,AD,AD"
4,002_S_0685,"[CN, CN, CN, CN, CN, CN, CN, CN]","[bl, m06, m12, m24, m36, m48, m60, m72]",2006-07-20,2012-07-27,72.247890,CN,CN,0,"CN,CN,CN,CN,CN,CN,CN,CN"
...,...,...,...,...,...,...,...,...,...,...
2401,941_S_7041,[MCI],[bl],2022-03-01,2022-03-01,0.000000,MCI,MCI,0,MCI
2402,941_S_7046,[CN],[bl],2022-04-06,2022-04-06,0.000000,CN,CN,0,CN
2403,941_S_7051,[CN],[bl],2022-03-30,2022-03-30,0.000000,CN,CN,0,CN
2404,941_S_7074,[CN],[bl],2022-05-02,2022-05-02,0.000000,CN,CN,0,CN


In [68]:
fullMerge = pd.merge(
    csv,
    csvMergePerPatient, # Nos quedamos solo con las columnas de interés
    how="left",
    on=None,
    left_on='Subject',
    right_on='PTID',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [69]:
fullMerge

,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
0,I240519,002_S_0295,CN,M,90,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2011-06-09,...,002_S_0295,"[CN, CN, CN, CN, CN, CN, CN, CN]","[bl, m06, m12, m24, m36, m48, m72, m60]",2006-05-09,2012-05-10,72.050761,CN,CN,0.0,"CN,CN,CN,CN,CN,CN,CN,CN"
1,I384689,002_S_0413,CN,F,83,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2013-05-14,...,002_S_0413,"[CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, C...","[bl, m06, m12, m24, m36, m48, m108, m96, m84, ...",2006-05-08,2019-08-22,159.477607,CN,CN,0.0,"CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN"
2,I241476,002_S_0413,CN,F,82,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2011-06-17,...,002_S_0413,"[CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, CN, C...","[bl, m06, m12, m24, m36, m48, m108, m96, m84, ...",2006-05-08,2019-08-22,159.477607,CN,CN,0.0,"CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN,CN"
3,I187801,002_S_0685,CN,F,94,m48,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2010-07-27,...,002_S_0685,"[CN, CN, CN, CN, CN, CN, CN, CN]","[bl, m06, m12, m24, m36, m48, m60, m72]",2006-07-20,2012-07-27,72.247890,CN,CN,0.0,"CN,CN,CN,CN,CN,CN,CN,CN"
4,I323915,002_S_0685,CN,F,96,v11,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2012-08-02,...,002_S_0685,"[CN, CN, CN, CN, CN, CN, CN, CN]","[bl, m06, m12, m24, m36, m48, m60, m72]",2006-07-20,2012-07-27,72.247890,CN,CN,0.0,"CN,CN,CN,CN,CN,CN,CN,CN"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,I1005488,941_S_6345,MCI,M,79,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2018-05-30,...,941_S_6345,"[MCI, AD, AD]","[bl, m12, m30]",2018-05-14,2020-08-31,27.598103,AD,AD,0.0,"MCI,AD,AD"
3758,I1239316,941_S_6803,MCI,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2019-10-07,...,941_S_6803,[MCI],[bl],2019-09-17,2019-09-17,0.000000,MCI,MCI,0.0,MCI
3759,I1301874,941_S_6854,AD,M,86,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2020-02-24,...,941_S_6854,"[AD, AD, AD]","[bl, m12, m24]",2020-03-04,2022-02-22,23.655517,AD,AD,0.0,"AD,AD,AD"
3760,I1480671,941_S_6962,AD,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2021-08-10,...,941_S_6962,"[AD, AD]","[bl, m12]",2021-07-13,2022-06-29,11.532064,AD,AD,0.0,"AD,AD"


In [70]:
fullMerge['DX_last']

0        CN
1        CN
2        CN
3        CN
4        CN
       ... 
3757     AD
3758    MCI
3759     AD
3760     AD
3761    MCI
Name: DX_last, Length: 3762, dtype: object

Ya tenemos el csv que queríamos con toda la data, pero sobran columnas

In [71]:
fullMerge.columns

Index(['Image Data ID', 'Subject', 'Group', 'Sex', 'Age', 'Visit', 'Modality',
       'Description', 'Type', 'Acq Date', 'Format', 'Downloaded', 'PTID', 'DX',
       'VISCODE', 'EXAMDATE_min', 'EXAMDATE_max', 'EXAMLENGTH', 'DX_last',
       'DX_most_severe', 'regressions', 'DX_hashed'],
      dtype='object')

In [72]:
len(fullMerge)

3762

In [73]:
print("En cuántos el grupo inicial es distinto al final:")
len(fullMerge[fullMerge['Group'] != fullMerge['DX_last']])

En cuántos el grupo inicial es distinto al final:


1786

In [74]:
csvMerge['VISCODE'].unique()

array(['bl', 'm06', 'm12', 'm24', 'm18', 'm36', 'm48', 'm60', 'm72',
       'm108', 'm96', 'm84', 'm120', 'm132', 'm144', 'm156', 'm168',
       'm180', 'm42', 'm174', 'm162', 'm138', 'm150', 'm102', 'm90',
       'm114', 'm78', 'm126', 'm66', 'm54', 'm30', 'm186', 'm192', 'm198'],
      dtype=object)

In [75]:
len(csvMerge['VISCODE'].unique())

34

In [76]:
csvMeta = pd.read_csv(petMetaCSV)
csvMeta['Visit'].unique()

array(['ADNI Baseline', 'ADNI1/GO Month 6', 'ADNI1/GO Month 12',
       'ADNI1/GO Month 18', 'ADNI1/GO Month 36', 'ADNI1/GO Month 24',
       'ADNI2 Year 1 Visit', 'ADNI2 Year 3 Visit', 'ADNI2 Year 5 Visit',
       'ADNI3 Initial Visit-Cont Pt', 'ADNI3 Year 2 Visit',
       'ADNI1/GO Month 48', 'ADNI2 Initial Visit-Cont Pt',
       'ADNI2 Year 2 Visit', 'ADNI2 Year 4 Visit', 'ADNIGO Month 60',
       'ADNI3 Year 1 Visit', 'ADNI3 Year 4 Visit',
       'ADNI2 Baseline-New Pt', 'ADNI2 Tau-only visit'], dtype=object)

In [77]:
len(csvMeta['Visit'].unique())

20

In [78]:
# TODO: revisar por qué el fullMerge si lo agrupo por pacientes me da 1600 pero el otro me da 2400 ??

In [79]:
fullMerge2 = fullMerge.copy()
fullMerge2['Acq Date'] = pd.to_datetime(fullMerge2['Acq Date'])
print("max acq date: ")
print(fullMerge2['Acq Date'].max())
print("min acq date: ")
print(fullMerge2['Acq Date'].min())
fullMerge2 = fullMerge2.groupby(by='Subject').agg({ 'Acq Date': [np.min, np.max] })

fullMerge2['Acq Date Length'] = (fullMerge2['Acq Date']['amax'] - fullMerge2['Acq Date']['amin'])/np.timedelta64(1, 'M')

# print the resulting DataFrame
fullMerge2

max acq date: 
2022-08-08 00:00:00
min acq date: 
2005-09-22 00:00:00


Acq Date            Acq Date Length
                 amin       amax                
Subject                                         
002_S_0295 2011-06-09 2011-06-09        0.000000
002_S_0413 2011-06-17 2013-05-14       22.899854
002_S_0685 2010-07-27 2012-08-02       24.214050
002_S_0729 2010-07-29 2012-08-14       24.542598
002_S_1155 2011-01-07 2017-04-28       75.664798
...               ...        ...             ...
941_S_6345 2018-05-30 2018-05-30        0.000000
941_S_6803 2019-10-07 2019-10-07        0.000000
941_S_6854 2020-02-24 2020-02-24        0.000000
941_S_6962 2021-08-10 2021-08-10        0.000000
941_S_7041 2022-03-09 2022-03-09        0.000000

[1687 rows x 3 columns]

In [80]:
fullMerge2['Acq Date Length'].mean()

18.918960139510563

In [81]:
fullMerge2['Acq Date Length'].std()

25.604222998200694

In [82]:
sum(fullMerge2['Acq Date Length'] == 0)

829

In [83]:
len(fullMerge2)

1687

In [84]:
fullMerge2.columns = ['Acq Date min', 'Acq date max', 'Acq Date Length']

La mitad tienen un solo estudio...

Ahora lo calculo por estudio:

In [85]:
fullMerge3 = pd.merge(fullMerge2, csv,     
    how="inner",
    on=None,
    left_on='Subject',
    right_on='Subject',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,)

In [86]:
fullMerge3

,Subject,Acq Date min,Acq date max,Acq Date Length,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,Format,Downloaded
0,002_S_0295,2011-06-09,2011-06-09,0.000000,I240519,CN,M,90,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2011-06-09,DCM,12/02/2022
1,002_S_0413,2011-06-17,2013-05-14,22.899854,I384689,CN,F,83,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2013-05-14,DCM,11/28/2022
2,002_S_0413,2011-06-17,2013-05-14,22.899854,I241476,CN,F,82,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2011-06-17,DCM,11/29/2022
3,002_S_0685,2010-07-27,2012-08-02,24.214050,I187801,CN,F,94,m48,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2010-07-27,DCM,11/28/2022
4,002_S_0685,2010-07-27,2012-08-02,24.214050,I323915,CN,F,96,v11,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2012-08-02,DCM,12/02/2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,941_S_6345,2018-05-30,2018-05-30,0.000000,I1005488,MCI,M,79,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2018-05-30,DCM,11/29/2022
3758,941_S_6803,2019-10-07,2019-10-07,0.000000,I1239316,MCI,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2019-10-07,DCM,11/28/2022
3759,941_S_6854,2020-02-24,2020-02-24,0.000000,I1301874,AD,M,86,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2020-02-24,DCM,12/02/2022
3760,941_S_6962,2021-08-10,2021-08-10,0.000000,I1480671,AD,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2021-08-10,DCM,1/09/2023


In [87]:
fullMerge3['Acq Date Length'].mean()

34.61900446535385

In [88]:
fullMerge3['Acq Date Length'].std()

31.219459558244065

# Armado del 80/10/10 DX_last

Tomando DX_last como categoría definitiva, primero dividimos los pacientes

In [89]:
csvAD = csvMergePerPatient[csvMergePerPatient['DX_last'] == 'AD'].copy(deep=True)
csvMCI = csvMergePerPatient[(csvMergePerPatient['DX_last'] == 'MCI')].copy(deep=True)
csvCN = csvMergePerPatient[csvMergePerPatient['DX_last'] == 'CN'].copy(deep=True)

In [90]:
csvAD

,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
3,002_S_0619,"[AD, AD, AD, AD]","[bl, m06, m12, m24]",2006-06-20,2008-08-07,25.593955,AD,AD,0,"AD,AD,AD,AD"
5,002_S_0729,"[MCI, MCI, AD, AD, AD, AD, AD, AD, AD, AD]","[bl, m06, m12, m36, m18, m24, m48, m84, m72, m60]",2006-08-02,2013-08-08,84.207068,AD,AD,0,"MCI,MCI,AD,AD,AD,AD,AD,AD,AD,AD"
7,002_S_0816,"[AD, AD, AD]","[m06, bl, m12]",2006-09-11,2008-01-28,16.558862,AD,AD,0,"AD,AD,AD"
8,002_S_0938,"[AD, AD, AD, AD]","[m24, bl, m06, m12]",2006-10-17,2008-10-23,24.214050,AD,AD,0,"AD,AD,AD,AD"
9,002_S_0954,"[MCI, MCI, AD, AD, AD]","[bl, m06, m18, m12, m24]",2006-11-16,2008-11-13,23.918356,AD,AD,0,"MCI,MCI,AD,AD,AD"
...,...,...,...,...,...,...,...,...,...,...
2365,941_S_4365,"[CN, CN, CN, CN, AD, MCI, MCI, AD]","[m24, m12, m06, bl, m114, m66, m78, m126]",2012-01-05,2022-08-10,127.148401,AD,AD,1,"CN,CN,CN,CN,AD,MCI,MCI,AD"
2377,941_S_6068,"[MCI, MCI, MCI, AD, AD]","[bl, m12, m24, m48, m60]",2017-08-30,2022-08-16,59.533050,AD,AD,0,"MCI,MCI,MCI,AD,AD"
2382,941_S_6345,"[MCI, AD, AD]","[bl, m12, m30]",2018-05-14,2020-08-31,27.598103,AD,AD,0,"MCI,AD,AD"
2398,941_S_6854,"[AD, AD, AD]","[bl, m12, m24]",2020-03-04,2022-02-22,23.655517,AD,AD,0,"AD,AD,AD"


In [91]:
len(csvAD)

681

In [92]:
len(csvMCI)

874

In [93]:
len(csvCN)

851

In [94]:
trainAD = math.floor(len(csvAD) * trainProportion)
valAD = math.floor(len(csvAD) * valProportion)
testAD = len(csvAD) - trainAD - valAD
print("trainAD", trainAD)
print("valAD", valAD)
print("testAD", testAD)

trainCN = math.floor(len(csvCN) * trainProportion)
valCN = math.floor(len(csvCN) * valProportion)
testCN = len(csvCN) - trainCN - valCN
print("trainCN", trainCN)
print("valCN", valCN)
print("testCN", testCN)

trainMCI = math.floor(len(csvMCI) * trainProportion)
valMCI = math.floor(len(csvMCI) * valProportion)
testMCI = len(csvMCI)  - trainMCI - valMCI
print("trainMCI", trainMCI)
print("valMCI", valMCI)
print("testMCI", testMCI)

trainAD 544
valAD 68
testAD 69
trainCN 680
valCN 85
testCN 86
trainMCI 699
valMCI 87
testMCI 88


In [95]:
# AD
trainADSample = csvAD.sample(n = trainAD, replace=False, random_state = randomState)
csvAD = csvAD.drop(trainADSample.index)
trainADSample.describe()

,EXAMLENGTH,regressions
count,544.000000,544.000000
mean,35.187520,0.106618
std,34.161283,0.337405
min,0.000000,0.000000
25%,12.222017,0.000000
50%,24.131912,0.000000
75%,39.688700,0.000000
max,191.806813,2.000000


In [96]:
len(trainADSample)

544

In [97]:
valADSample = csvAD.sample(n = valAD, replace=False, random_state = randomState)
csvAD = csvAD.drop(valADSample.index)
valADSample.describe()
len(valADSample)

68

In [98]:
# Test
len(csvAD)

69

In [99]:
# CN
trainCNSample = csvCN.sample(n = trainCN, replace=False, random_state = randomState)
csvCN = csvCN.drop(trainCNSample.index)
print(len(trainCNSample))

valCNSample = csvCN.sample(n = valCN, replace=False, random_state = randomState)
csvCN = csvCN.drop(valCNSample.index)
print(len(valCNSample))
len(csvCN)

680
85


86

In [100]:
# MCI
trainMCISample = csvMCI.sample(n = trainMCI, replace=False, random_state = randomState)
csvMCI = csvMCI.drop(trainMCISample.index)
print(len(trainMCISample))

valMCISample = csvMCI.sample(n = valMCI, replace=False, random_state = randomState)
csvMCI = csvMCI.drop(valMCISample.index)
print(len(valMCISample))
len(csvMCI)

699
87


88

In [101]:
trainMCISample.columns

Index(['PTID', 'DX', 'VISCODE', 'EXAMDATE_min', 'EXAMDATE_max', 'EXAMLENGTH',
       'DX_last', 'DX_most_severe', 'regressions', 'DX_hashed'],
      dtype='object')

In [102]:
trainCSV = pd.DataFrame(columns=csvMergePerPatient.columns)
valCSV = pd.DataFrame(columns=csvMergePerPatient.columns)
testCSV = pd.DataFrame(columns=csvMergePerPatient.columns)

In [103]:
def addIfInDxClasses(array, sample, klass):
    if klass in dxClasses:
        array.append(sample)
    else:
        print(f"Ignoring {klass} samples")

In [104]:
concatArray = []
addIfInDxClasses(concatArray, trainADSample, "AD")
addIfInDxClasses(concatArray, trainCNSample, "CN")
addIfInDxClasses(concatArray, trainMCISample, "MCI")
    
trainCSV = pd.concat(concatArray, ignore_index = True)
len(trainCSV)

1923

In [105]:
concatArray = []
addIfInDxClasses(concatArray, valADSample, "AD")
addIfInDxClasses(concatArray, valCNSample, "CN")
addIfInDxClasses(concatArray, valMCISample, "MCI")

valCSV = pd.concat(concatArray, ignore_index = True)
len(valCSV)

240

In [106]:
concatArray = []
addIfInDxClasses(concatArray, csvAD, "AD")
addIfInDxClasses(concatArray, csvCN, "CN")
addIfInDxClasses(concatArray, csvMCI, "MCI")

testCSV = pd.concat(concatArray, ignore_index = True)
len(testCSV)

243

Pero los sets por ahora no tienen la data de las imágenes, solo de los subjects. Hay que mergear

In [107]:
trainCSVMerge = pd.merge(
    csv,
    trainCSV,
    how="inner",
    on=None,
    left_on='Subject',
    right_on='PTID',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [108]:
print("train AD: ", len(trainCSVMerge[trainCSVMerge['DX_last']== 'AD']))
print("train MCI: ", len(trainCSVMerge[trainCSVMerge['DX_last']== 'MCI']))
print("train CN: ", len(trainCSVMerge[trainCSVMerge['DX_last']== 'CN']))

train AD:  1039
train MCI:  1229
train CN:  765


In [109]:
len(trainCSVMerge)

3033

In [110]:
valCSVMerge = pd.merge(
    csv,
    valCSV, # Nos quedamos solo con las columnas de interés
    how="inner",
    on=None,
    left_on='Subject',
    right_on='PTID',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

print("val AD: ", len(valCSVMerge[valCSVMerge['DX_last']== 'AD']))
print("val MCI: ", len(valCSVMerge[valCSVMerge['DX_last']== 'MCI']))
print("val CN: ", len(valCSVMerge[valCSVMerge['DX_last']== 'CN']))

val AD:  126
val MCI:  145
val CN:  86


In [111]:
len(valCSVMerge)

357

In [112]:
testCSVMerge = pd.merge(
    csv,
    testCSV, # Nos quedamos solo con las columnas de interés
    how="inner",
    on=None,
    left_on='Subject',
    right_on='PTID',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)


print("test AD: ", len(testCSVMerge[testCSVMerge['DX_last']== 'AD']))
print("test MCI: ", len(testCSVMerge[testCSVMerge['DX_last']== 'MCI']))
print("test CN: ", len(testCSVMerge[testCSVMerge['DX_last']== 'CN']))

test AD:  113
test MCI:  149
test CN:  97


In [113]:
len(testCSVMerge)

359

In [114]:
trainCSVMerge.to_csv(dxLastTrainOutputFile)
valCSVMerge.to_csv(dxLastValOutputFile)
testCSVMerge.to_csv(dxLastTestOutputFile)

# DX_last train mini

In [115]:
trainCSVMergeMini = trainCSVMerge.copy()

In [116]:
trainCSVMergeMini = trainCSVMergeMini.drop(trainCSVMergeMini[trainCSVMergeMini['DX_last'] == 'AD'].sample(frac=.9).index)

In [117]:
trainCSVMergeMini = trainCSVMergeMini.drop(trainCSVMergeMini[trainCSVMergeMini['DX_last'] == 'MCI'].sample(frac=.9).index)

In [118]:
trainCSVMergeMini = trainCSVMergeMini.drop(trainCSVMergeMini[trainCSVMergeMini['DX_last'] == 'CN'].sample(frac=.9).index)

In [119]:
trainCSVMergeMini.groupby(by='DX_last').describe()

Age                                                    EXAMLENGTH  \
         count       mean       std   min   25%   50%   75%   max      count   
DX_last                                                                        
AD       104.0  75.711538  7.517840  56.0  70.0  76.0  81.0  90.0      104.0   
CN        77.0  75.532468  6.208196  56.0  73.0  76.0  80.0  86.0       77.0   
MCI      123.0  75.170732  7.452260  55.0  70.0  75.0  81.0  89.0      123.0   

                    ...                         regressions            \
              mean  ...         75%         max       count      mean   
DX_last             ...                                                 
AD       57.166866  ...   83.738886  160.561819       104.0  0.134615   
CN       72.319147  ...  103.985708  180.767572        77.0  0.259740   
MCI      59.910480  ...   85.143432  192.299637       123.0  0.235772   

                                            
              std  min  25%  50%  75%  max  
DX_last                                     
AD       0.370193  0.0  0.0  0.0  0.0  2.0  
CN       0.547785  0.0  0.0  0.0  0.0  2.0  
MCI      0.445033  0.0  0.0  0.0  0.0  2.0  

[3 rows x 24 columns]

In [120]:
trainCSVMergeMini.to_csv(dxLastTrainMiniOutputFile)

# 80/10/10 DX_most_severe

In [121]:
csvAD = csvMergePerPatient[csvMergePerPatient['DX_most_severe'] == 'AD'].copy(deep=True)
csvMCI = csvMergePerPatient[(csvMergePerPatient['DX_most_severe'] == 'MCI')].copy(deep=True)
csvCN = csvMergePerPatient[csvMergePerPatient['DX_most_severe'] == 'CN'].copy(deep=True)

print("Pacientes: ")
len(csvAD)

len(csvMCI)

len(csvCN)

trainAD = math.floor(len(csvAD) * trainProportion)
valAD = math.floor(len(csvAD) * valProportion)
testAD = len(csvAD) - trainAD - valAD
print("trainAD", trainAD)
print("valAD", valAD)
print("testAD", testAD)

trainCN = math.floor(len(csvCN) * trainProportion)
valCN = math.floor(len(csvCN) * valProportion)
testCN = len(csvCN) - trainCN - valCN
print("trainCN", trainCN)
print("valCN", valCN)
print("testCN", testCN)

trainMCI = math.floor(len(csvMCI) * trainProportion)
valMCI = math.floor(len(csvMCI) * valProportion)
testMCI = len(csvMCI)  - trainMCI - valMCI
print("trainMCI", trainMCI)
print("valMCI", valMCI)
print("testMCI", testMCI)

# AD
trainADSample = csvAD.sample(n = trainAD, replace=False, random_state = randomState)
csvAD = csvAD.drop(trainADSample.index)
trainADSample.describe()

len(trainADSample)

valADSample = csvAD.sample(n = valAD, replace=False, random_state = randomState)
csvAD = csvAD.drop(valADSample.index)
valADSample.describe()
len(valADSample)

# Test
len(csvAD)

# CN
trainCNSample = csvCN.sample(n = trainCN, replace=False, random_state = randomState)
csvCN = csvCN.drop(trainCNSample.index)
print(len(trainCNSample))

valCNSample = csvCN.sample(n = valCN, replace=False, random_state = randomState)
csvCN = csvCN.drop(valCNSample.index)
print(len(valCNSample))
len(csvCN)

# MCI
trainMCISample = csvMCI.sample(n = trainMCI, replace=False, random_state = randomState)
csvMCI = csvMCI.drop(trainMCISample.index)
print(len(trainMCISample))

valMCISample = csvMCI.sample(n = valMCI, replace=False, random_state = randomState)
csvMCI = csvMCI.drop(valMCISample.index)
print(len(valMCISample))
len(csvMCI)

trainMCISample.columns

trainCSV = pd.DataFrame(columns=csvMergePerPatient.columns)
valCSV = pd.DataFrame(columns=csvMergePerPatient.columns)
testCSV = pd.DataFrame(columns=csvMergePerPatient.columns)

Pacientes: 
trainAD 660
valAD 82
testAD 83
trainCN 596
valCN 74
testCN 76
trainMCI 668
valMCI 83
testMCI 84
596
74
668
83


In [122]:
concatArray = []
addIfInDxClasses(concatArray, trainADSample, "AD")
addIfInDxClasses(concatArray, trainCNSample, "CN")
addIfInDxClasses(concatArray, trainMCISample, "MCI")

trainCSV = pd.concat(concatArray, ignore_index = True)
len(trainCSV)

concatArray = []
addIfInDxClasses(concatArray, valADSample, "AD")
addIfInDxClasses(concatArray, valCNSample, "CN")
addIfInDxClasses(concatArray, valMCISample, "MCI")
valCSV = pd.concat(concatArray, ignore_index = True)
len(valCSV)

concatArray = []
addIfInDxClasses(concatArray, csvAD, "AD")
addIfInDxClasses(concatArray, csvCN, "CN")
addIfInDxClasses(concatArray, csvMCI, "MCI")

testCSV = pd.concat(concatArray, ignore_index = True)
len(testCSV)

# Pero los sets por ahora no tienen la data de las imágenes, solo de los subjects. Hay que mergear
print("Imágenes:")
trainCSVMerge = pd.merge(
    csv,
    trainCSV, # Nos quedamos solo con las columnas de interés
    how="inner",
    on=None,
    left_on='Subject',
    right_on='PTID',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

print("train AD: ", len(trainCSVMerge[trainCSVMerge['DX_most_severe']== 'AD']))
print("train MCI: ", len(trainCSVMerge[trainCSVMerge['DX_most_severe']== 'MCI']))
print("train CN: ", len(trainCSVMerge[trainCSVMerge['DX_most_severe']== 'CN']))

len(trainCSVMerge)

valCSVMerge = pd.merge(
    csv,
    valCSV, # Nos quedamos solo con las columnas de interés
    how="inner",
    on=None,
    left_on='Subject',
    right_on='PTID',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

print("val AD: ", len(valCSVMerge[valCSVMerge['DX_most_severe']== 'AD']))
print("val MCI: ", len(valCSVMerge[valCSVMerge['DX_most_severe']== 'MCI']))
print("val CN: ", len(valCSVMerge[valCSVMerge['DX_most_severe']== 'CN']))

len(valCSVMerge)

testCSVMerge = pd.merge(
    csv,
    testCSV, # Nos quedamos solo con las columnas de interés
    how="inner",
    on=None,
    left_on='Subject',
    right_on='PTID',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)


print("test AD: ", len(testCSVMerge[testCSVMerge['DX_most_severe']== 'AD']))
print("test MCI: ", len(testCSVMerge[testCSVMerge['DX_most_severe']== 'MCI']))
print("test CN: ", len(testCSVMerge[testCSVMerge['DX_most_severe']== 'CN']))

len(testCSVMerge)

trainCSVMerge.to_csv(dxMostSevereTrainOutputFile)
valCSVMerge.to_csv(dxMostSevereValOutputFile)
testCSVMerge.to_csv(dxMostSevereTestOutputFile)

Imágenes:
train AD:  1249
train MCI:  1173
train CN:  590
val AD:  163
val MCI:  144
val CN:  58
test AD:  189
test MCI:  107
test CN:  76


# 80/10/10 por visita

In [123]:
# Este es un poco distinto a los anteriores, usamos dos archivos, y matcheamos por el SubjectID/PTID y el mes.
# Para eso, tenemos que convertir el campo Visit 
# La forma en que esto funciona:
# Desde el csv, agarramos el subject id y el visit
# Hacemos join con DXSUM_... con esos dos campos para obtener el visitcode2, que es como está puesto en adnimerge
# Una vez que obtuvimos el visitcode2, mergeamos con ADNI merge, y en el campo DX tenemos el diagnóstico
# csv
# csvDXSUM
# csvMerge
csvDXSUM = pd.read_csv(csvDXSUMFile)

In [124]:
csvDXSUM['VISCODE2'].unique()

array(['bl', 'm06', 'm12', 'm24', 'm18', 'uns1', 'm36', 'm48', 'm60',
       'm72', 'sc', 'm84', 'm96', 'm108', 'm120', 'm132', 'm54', 'm66',
       'm78', 'm126', 'm42', 'm138', 'm144', 'm90', 'm150', 'm102',
       'm156', 'm162', nan, 'm30', 'm168', 'm114', 'm174', 'm180', 'm186',
       'm192', 'm198'], dtype=object)

In [125]:
csvMergePerVisit = pd.merge(
    csv,
    csvDXSUM,
    how="inner",
    on=None,
    left_on=['Subject', 'Visit'],
    right_on=['PTID', 'VISCODE'],
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [126]:
len(csvMergePerVisit)

3746

In [127]:
csvMergePerVisit['VISCODE2'].unique()

array(['m60', 'm84', 'm48', 'm72', 'm126', 'bl', 'm24', 'm132', 'm06',
       'm12', 'm36', 'm18', 'm66', 'm78', 'm54', 'm96', 'm138', 'm120',
       'm90', 'm144', 'm150', 'm108'], dtype=object)

In [128]:
sum(csvMergePerVisit['VISCODE2'].isna())

0

Hay 16 que no tienen diagnóstico. Como son pocos, los eliminamos...

In [129]:
csvMergePerVisitFinal = pd.merge(
    csvMergePerVisit,
    csvMerge,
    how="inner",
    on=None,
    left_on=['Subject', 'VISCODE2'],
    right_on=['PTID', 'VISCODE'],
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [130]:
len(csvMergePerVisitFinal)

3744

In [131]:
len(csvMergePerVisitFinal['Image Data ID'].unique())

3744

In [132]:
sum(csvMergePerVisitFinal['Subject'].isna())

0

In [133]:
len(csvMergePerVisitFinal['Subject'].unique())

1674

In [134]:
csvMergePerVisitFinal['DX'].unique()

array(['CN', 'AD', 'MCI'], dtype=object)

In [135]:
sum(csvMergePerVisitFinal['DX'].isna())

0

Eliminamos las clases que no nos interesan

In [136]:
csvMergePerVisitFinal.groupby(by='DX').describe()

Age                                                         ID  \
      count       mean       std   min   25%   50%   75%   max   count   
DX                                                                       
AD    861.0  76.405343  7.421021  55.0  72.0  77.0  82.0  96.0   762.0   
CN   1045.0  75.913876  6.251456  56.0  72.0  76.0  80.0  96.0  1025.0   
MCI  1838.0  74.828074  7.760629  55.0  70.0  75.0  81.0  94.0  1558.0   

                  ... Month              M                                  \
            mean  ...   75%    max   count       mean        std  min  25%   
DX                ...                                                        
AD   4100.918635  ...  24.0  138.0   861.0  18.648084  23.799413  0.0  0.0   
CN   3967.467317  ...  24.0  132.0  1045.0  18.884211  22.846795  0.0  0.0   
MCI  3525.863928  ...  24.0  150.0  1838.0  15.045702  22.945275  0.0  0.0   

                        
      50%   75%    max  
DX                      
AD   12.0  24.0  138.0  
CN   12.0  24.0  132.0  
MCI   6.0  24.0  150.0  

[3 rows x 1008 columns]

In [137]:
csvMergePerVisitFinal = csvMergePerVisitFinal[csvMergePerVisitFinal['DX'].isin(dxClasses)]

In [138]:
csvMergePerVisitFinal.groupby(by='DX').describe()

Age                                                         ID  \
      count       mean       std   min   25%   50%   75%   max   count   
DX                                                                       
AD    861.0  76.405343  7.421021  55.0  72.0  77.0  82.0  96.0   762.0   
CN   1045.0  75.913876  6.251456  56.0  72.0  76.0  80.0  96.0  1025.0   
MCI  1838.0  74.828074  7.760629  55.0  70.0  75.0  81.0  94.0  1558.0   

                  ... Month              M                                  \
            mean  ...   75%    max   count       mean        std  min  25%   
DX                ...                                                        
AD   4100.918635  ...  24.0  138.0   861.0  18.648084  23.799413  0.0  0.0   
CN   3967.467317  ...  24.0  132.0  1045.0  18.884211  22.846795  0.0  0.0   
MCI  3525.863928  ...  24.0  150.0  1838.0  15.045702  22.945275  0.0  0.0   

                        
      50%   75%    max  
DX                      
AD   12.0  24.0  138.0  
CN   12.0  24.0  132.0  
MCI   6.0  24.0  150.0  

[3 rows x 1008 columns]

In [139]:
csvMergePerVisitFinal.columns

Index(['Image Data ID', 'Subject', 'Group', 'Sex', 'Age', 'Visit', 'Modality',
       'Description', 'Type', 'Acq Date',
       ...
       'PTAU_bl', 'FDG_bl', 'PIB_bl', 'AV45_bl', 'FBB_bl', 'Years_bl',
       'Month_bl', 'Month', 'M', 'update_stamp_y'],
      dtype='object', length=182)

In [140]:
csvMergePerVisitFinal['EXAMDATE_x']

0       2011-06-02
1       2011-06-16
2       2013-05-14
3       2010-07-15
4       2012-07-24
           ...    
3739    2018-05-14
3740    2019-09-17
3741    2020-03-04
3742    2021-07-13
3743    2022-02-09
Name: EXAMDATE_x, Length: 3744, dtype: object

In [141]:
# Check for exam dates
csvMergePerVisitFinalCheck = csvMergePerVisitFinal.copy()
csvMergePerVisitFinalCheck['EXAMDATE'] = pd.to_datetime(csvMergePerVisitFinalCheck['EXAMDATE_x'])
# group by PTID and calculate the minimum and maximum EXAMDATE for each group
csvMergePerVisitFinalCheck = csvMergePerVisitFinalCheck.groupby(by='Subject').agg({'EXAMDATE': [np.min, np.max]})

# calculate the length in months between the first and last EXAMDATE for each patient
csvMergePerVisitFinalCheck['EXAMLENGTH'] = (csvMergePerVisitFinalCheck['EXAMDATE']['amax'] - csvMergePerVisitFinalCheck['EXAMDATE']['amin'])/np.timedelta64(1, 'M')

# print the resulting DataFrame
print("Exam length mean:")
print(csvMergePerVisitFinalCheck['EXAMLENGTH'].mean())
print("Exam length std:")
print(csvMergePerVisitFinalCheck['EXAMLENGTH'].std())
print("Exam length max:")
print(csvMergePerVisitFinalCheck['EXAMLENGTH'].max())
print("Images with only 1 exam:")
print(sum(csvMergePerVisitFinalCheck['EXAMLENGTH'] == 0))

Exam length mean:
19.22303329648819
Exam length std:
25.802980077793478
Exam length max:
153.00519517854576
Images with only 1 exam:
816


In [142]:
csvMergePerVisitFinalCheck.columns

MultiIndex([(  'EXAMDATE', 'amin'),
            (  'EXAMDATE', 'amax'),
            ('EXAMLENGTH',     '')],
           )

In [143]:
csvMergePerVisitFinalCheck.columns = ['examdate min', 'examdate max', 'EXAMLENGTH']

In [144]:
csvMergePerVisitFinalCheck = pd.merge(
    csvMergePerVisitFinalCheck, csv, 
    how="inner",
    on=None,
    left_on='Subject',
    right_on='Subject',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [145]:
csvMergePerVisitFinalCheck['EXAMLENGTH']

0        0.000000
1       22.932709
2       22.932709
3       24.312614
4       24.312614
          ...    
3744     0.000000
3745     0.000000
3746     0.000000
3747     0.000000
3748     0.000000
Name: EXAMLENGTH, Length: 3749, dtype: float64

In [146]:
print(csvMergePerVisitFinalCheck['EXAMLENGTH'].mean())
print(csvMergePerVisitFinalCheck['EXAMLENGTH'].std())
print(csvMergePerVisitFinalCheck['EXAMLENGTH'].max())
print(sum(csvMergePerVisitFinalCheck['EXAMLENGTH'] == 0))

34.99919122899072
31.441379232538456
153.00519517854576
817


Volviendo a lo importante...

In [147]:
csvMergePerVisitFinal = csvMergePerVisitFinal[['Image Data ID', 'Subject', 'DX', 'Group', 'Sex', 'Age', 'Visit', 'Modality',
       'Description', 'Type', 'Acq Date', 'VISCODE2']]

In [148]:
csvMergePerVisitFinal['DX']

0        CN
1        CN
2        CN
3        CN
4        CN
       ... 
3739    MCI
3740    MCI
3741     AD
3742     AD
3743    MCI
Name: DX, Length: 3744, dtype: object

Una vez creado el csv correcto es hora de crear los distintos sets...

In [149]:
patients = csv.groupby(by='Subject').groups.keys()
csvPatient = pd.DataFrame(patients, columns = ['Subject'])

In [150]:
csvPatient

,Subject
0,002_S_0295
1,002_S_0413
2,002_S_0685
3,002_S_0729
4,002_S_1155
...,...
1682,941_S_6345
1683,941_S_6803
1684,941_S_6854
1685,941_S_6962


In [151]:
len(csvPatient)

1687

In [152]:
trainN = round(trainProportion * len(csvPatient))
valN = round(valProportion * len(csvPatient))
testN = len(csvPatient) - trainN - valN

In [153]:
print(trainN)
print(valN)
print(testN)

1350
169
168


In [154]:
trainSample = csvPatient.sample(n = trainN, replace=False, random_state = randomState)
csvPatient = csvPatient.drop(trainSample.index)
trainSample = trainSample.reset_index()
trainSample = trainSample[['Subject']]
len(trainSample)

1350

In [155]:
valSample = csvPatient.sample(n = valN, replace=False, random_state = randomState)
csvPatient = csvPatient.drop(valSample.index)
valSample = valSample.reset_index()
valSample = valSample[['Subject']]
len(valSample)

169

In [156]:
testSample = csvPatient.sample(n = testN, replace=False, random_state = randomState)
csvPatient = csvPatient.drop(testSample.index)
testSample = testSample.reset_index()
testSample = testSample[['Subject']]
len(testSample)

168

In [157]:
len(csvPatient)

0

In [158]:
trainSampleMerge = pd.merge(
    csvMergePerVisitFinal,
    trainSample,
    how="inner",
    on=None,
    left_on=['Subject'],
    right_on=['Subject'],
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [159]:
len(trainSampleMerge)

3019

In [160]:
len(trainSampleMerge['Image Data ID'].unique())

3019

In [161]:
trainSampleMerge.groupby(by='Subject').count().std()

Image Data ID    1.823624
DX               1.823624
Group            1.823624
Sex              1.823624
Age              1.823624
Visit            1.823624
Modality         1.823624
Description      1.823624
Type             1.823624
Acq Date         1.823624
VISCODE2         1.823624
dtype: float64

In [162]:
trainSampleMerge.groupby(by='DX').describe()

Age                                                   
      count       mean       std   min   25%   50%   75%   max
DX                                                            
AD    687.0  76.515284  7.610756  56.0  72.0  77.0  82.0  96.0
CN    837.0  75.794504  6.428727  56.0  71.0  76.0  80.0  96.0
MCI  1495.0  74.981940  7.657516  55.0  70.0  75.0  81.0  92.0

In [163]:
valSampleMerge = pd.merge(
    csvMergePerVisitFinal,
    valSample,
    how="inner",
    on=None,
    left_on=['Subject'],
    right_on=['Subject'],
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

In [164]:
len(valSampleMerge)

357

In [165]:
len(valSampleMerge['Image Data ID'].unique())

357

In [166]:
valSampleMerge.groupby(by='Subject').count().std()

Image Data ID    1.682354
DX               1.682354
Group            1.682354
Sex              1.682354
Age              1.682354
Visit            1.682354
Modality         1.682354
Description      1.682354
Type             1.682354
Acq Date         1.682354
VISCODE2         1.682354
dtype: float64

In [167]:
valSampleMerge.groupby(by='DX').describe()

Age                                                    
     count       mean       std   min    25%   50%   75%   max
DX                                                            
AD    90.0  75.644444  7.224041  56.0  72.00  75.0  81.0  92.0
CN    89.0  77.359551  6.232698  62.0  74.00  77.0  81.0  94.0
MCI  178.0  75.511236  8.197587  55.0  69.25  76.0  82.0  94.0

In [168]:
testSampleMerge = pd.merge(
    csvMergePerVisitFinal,
    testSample,
    how="inner",
    on=None,
    left_on=['Subject'],
    right_on=['Subject'],
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)
len(testSampleMerge)

368

In [169]:
len(testSampleMerge['Image Data ID'].unique())

368

In [170]:
testSampleMerge.groupby(by='Subject').count().std()

Image Data ID    1.716573
DX               1.716573
Group            1.716573
Sex              1.716573
Age              1.716573
Visit            1.716573
Modality         1.716573
Description      1.716573
Type             1.716573
Acq Date         1.716573
VISCODE2         1.716573
dtype: float64

In [171]:
testSampleMerge.groupby(by='DX').describe()

Age                                                   
     count       mean       std   min   25%   50%   75%   max
DX                                                           
AD    84.0  76.321429  5.931660  55.0  73.0  77.0  80.0  87.0
CN   119.0  75.672269  4.717636  63.0  73.0  75.0  78.0  90.0
MCI  165.0  72.696970  7.913730  56.0  67.0  73.0  78.0  89.0

In [172]:
len(trainSampleMerge) + len(valSampleMerge) + len(testSampleMerge)

3744

In [173]:
# Chequeamos proporciones

In [174]:
trainSampleMerge.to_csv(dxVisitTrainOutputFile)
valSampleMerge.to_csv(dxVisitValOutputFile)
testSampleMerge.to_csv(dxVisitTestOutputFile)

In [175]:
# 3744 son menos que el total de imágenes pero esto es porque en los merge tenían NAN en algún campo, 
# por ende se descartan

In [176]:
df = pd.concat([trainSampleMerge, valSampleMerge, testSampleMerge])

In [177]:
len(df)

3744

In [178]:
len(df['Image Data ID'])

3744

In [179]:
len(df['Image Data ID'].unique())

3744

# 80/10/10 visit CN vs AD nuevo

En este nuevo set tiene que:
a) O tener una sola imágen por paciente.
b) O varias imágenes por pacientes pero:
No tener los MCI.
No tener los CN/MCI.

In [180]:
csvMergePerVisitFinal

,Image Data ID,Subject,DX,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,VISCODE2
0,I240519,002_S_0295,CN,CN,M,90,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2011-06-09,m60
1,I241476,002_S_0413,CN,CN,F,82,v06,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2011-06-17,m60
2,I384689,002_S_0413,CN,CN,F,83,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2013-05-14,m84
3,I187801,002_S_0685,CN,CN,F,94,m48,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2010-07-27,m48
4,I323915,002_S_0685,CN,CN,F,96,v11,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2012-08-02,m72
...,...,...,...,...,...,...,...,...,...,...,...,...
3739,I1005488,941_S_6345,MCI,MCI,M,79,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2018-05-30,bl
3740,I1239316,941_S_6803,MCI,MCI,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2019-10-07,bl
3741,I1301874,941_S_6854,AD,AD,M,86,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2020-02-24,bl
3742,I1480671,941_S_6962,AD,AD,F,75,bl,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,2021-08-10,bl


In [181]:
csvm = csvMergePerVisitFinal.groupby("Subject").aggregate({"DX": list, "Image Data ID": "count"})
csvm

,DX,Image Data ID
Subject,,
002_S_0295,[CN],1
002_S_0413,"[CN, CN]",2
002_S_0685,"[CN, CN]",2
002_S_0729,"[AD, AD]",2
002_S_1155,"[MCI, MCI, MCI]",3
...,...,...
941_S_6345,[MCI],1
941_S_6803,[MCI],1
941_S_6854,[AD],1


In [182]:
csvm[csvm['Image Data ID'] > 3]

,DX,Image Data ID
Subject,,
003_S_1057,"[MCI, MCI, MCI, AD, AD, AD, AD]",7
003_S_1059,"[AD, AD, AD, AD]",4
003_S_1074,"[MCI, MCI, MCI, AD, MCI, MCI, MCI, MCI, MCI]",9
003_S_1122,"[MCI, MCI, MCI, MCI, MCI, MCI, MCI, MCI, MCI, ...",10
005_S_0221,"[AD, AD, AD, AD]",4
...,...,...
941_S_1197,"[CN, CN, CN, CN]",4
941_S_1202,"[CN, CN, CN, MCI, MCI, MCI]",6
941_S_1203,"[CN, CN, CN, MCI]",4


Creamos una nueva etiqueta DX_vis2 para poner ahí la categoría nueva que se basa en: 
Si tiene solamente de una categoría, es esa categoría.
Si tiene AD en algún momento, es AD.
Si tiene CN en algún momento, es no-AD (fue CN alguna vez).
Si tiene solo MCI, entonces es MCI.

Y se diferencia entre categorías "puras" versus categorías mixtas ('AD', 'no-AD').

In [183]:
def process_funct(x):
    
    allEqual = True
    first = x['DX'][0]
    hasAD = False
    hasCN = False
    for y in x['DX']:
        if y != first:
            allEqual = False
        if y == 'AD':
            hasAD = True
        if y == 'MCI':
            hasCN = True
            
            
    if allEqual == True:
        return f"{first}-puro"
    
    if hasAD:
        return 'AD'
    
    if hasCN:
        return 'no-AD'
    
    return 'MCI'

# vis2 es el nuevo
csvm['DX_vis2'] = csvm.apply(process_funct, axis = 1)
csvm

,DX,Image Data ID,DX_vis2
Subject,,,
002_S_0295,[CN],1,CN-puro
002_S_0413,"[CN, CN]",2,CN-puro
002_S_0685,"[CN, CN]",2,CN-puro
002_S_0729,"[AD, AD]",2,AD-puro
002_S_1155,"[MCI, MCI, MCI]",3,MCI-puro
...,...,...,...
941_S_6345,[MCI],1,MCI-puro
941_S_6803,[MCI],1,MCI-puro
941_S_6854,[AD],1,AD-puro


In [184]:
csvm.groupby('DX_vis2').describe()

Image Data ID                                              
                 count      mean       std  min  25%  50%  75%   max
DX_vis2                                                             
AD               155.0  4.632258  2.180252  2.0  2.0  5.0  6.0   9.0
AD-puro          331.0  1.785498  1.167450  1.0  1.0  1.0  2.0   6.0
CN-puro          398.0  2.108040  1.640448  1.0  1.0  2.0  2.0   8.0
MCI-puro         721.0  1.852982  1.525895  1.0  1.0  1.0  2.0  10.0
no-AD             69.0  3.768116  1.941383  2.0  2.0  3.0  6.0   8.0

Nos quedan 486 sujetos con AD, 467 sujetos con no-AD. Si solo consideraramos una imágen por sujeto, quedarían 953 imágenes. Faltaría contar cuántas imágenes tenemos para cada grupo:

In [185]:
csvm.groupby('DX_vis2').aggregate({"Image Data ID": 'sum'})

,Image Data ID
DX_vis2,
AD,718
AD-puro,591
CN-puro,839
MCI-puro,1336
no-AD,260


Es decir, tenemos 155.0 ADDs "impuros", 591 imágenes de AD "puro" (contando varios estudios de la misma persona), 839 imágenes de CNs "puros", y 69.0 imágenes "no-AD" (CN -> MCI). En total, no usando las imágenes de MCI-puro, tendríamos:
1654 imágenes.

Notar que para los casos mixtos (AD y no-AD) estoy usando la tabla anterior: 1 por paciente.

In [186]:
csvm.groupby(['DX_vis2']).aggregate({"Image Data ID": ['sum', 'mean', 'std']})

Image Data ID                    
                   sum      mean       std
DX_vis2                                   
AD                 718  4.632258  2.180252
AD-puro            591  1.785498  1.167450
CN-puro            839  2.108040  1.640448
MCI-puro          1336  1.852982  1.525895
no-AD              260  3.768116  1.941383

Veamos csvmMCI si podemos rescatar algunos:

In [187]:
csvmMCI = csvm[csvm['DX_vis2'] == "MCI-puro"]
csvmMCI

,DX,Image Data ID,DX_vis2
Subject,,,
002_S_1155,"[MCI, MCI, MCI]",3,MCI-puro
002_S_2043,[MCI],1,MCI-puro
002_S_2073,"[MCI, MCI]",2,MCI-puro
002_S_4219,[MCI],1,MCI-puro
002_S_4229,"[MCI, MCI, MCI]",3,MCI-puro
...,...,...,...
941_S_6052,[MCI],1,MCI-puro
941_S_6068,[MCI],1,MCI-puro
941_S_6345,[MCI],1,MCI-puro


In [188]:
csvmMCI[csvmMCI['Image Data ID'] >= 4]

,DX,Image Data ID,DX_vis2
Subject,,,
003_S_1122,"[MCI, MCI, MCI, MCI, MCI, MCI, MCI, MCI, MCI, ...",10,MCI-puro
005_S_0546,"[MCI, MCI, MCI, MCI, MCI, MCI, MCI]",7,MCI-puro
007_S_0414,"[MCI, MCI, MCI, MCI, MCI]",5,MCI-puro
007_S_0698,"[MCI, MCI, MCI, MCI, MCI, MCI, MCI]",7,MCI-puro
012_S_0634,"[MCI, MCI, MCI, MCI, MCI, MCI]",6,MCI-puro
...,...,...,...
137_S_0158,"[MCI, MCI, MCI, MCI]",4,MCI-puro
137_S_0481,"[MCI, MCI, MCI, MCI, MCI, MCI]",6,MCI-puro
137_S_0800,"[MCI, MCI, MCI, MCI, MCI, MCI, MCI, MCI]",8,MCI-puro


In [189]:
csvmMCI['Image Data ID'].describe()

count    721.000000
mean       1.852982
std        1.525895
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max       10.000000
Name: Image Data ID, dtype: float64

Pero aun así no debería usarlos porque son MCI, no sabemos cómo van a evolucionar ?

### visit_953

In [190]:
visit953 = csvm[csvm['DX_vis2'].isin(['CN-puro', 'AD-puro', 'AD', 'no-AD'])].copy()

In [191]:
visit953

,DX,Image Data ID,DX_vis2
Subject,,,
002_S_0295,[CN],1,CN-puro
002_S_0413,"[CN, CN]",2,CN-puro
002_S_0685,"[CN, CN]",2,CN-puro
002_S_0729,"[AD, AD]",2,AD-puro
002_S_1261,"[CN, CN]",2,CN-puro
...,...,...,...
941_S_4376,[CN],1,CN-puro
941_S_5124,[CN],1,CN-puro
941_S_5193,[CN],1,CN-puro


Una vez que tenemos los subjects y su categoría, tenemos también que obtener la imágen específica.

In [192]:
visit953Img = pd.merge(
    visit953[['DX_vis2']],
    csvMergePerVisitFinal[['Subject', 'Image Data ID', 'Sex', 'Age', 'Visit', 'DX']],
    how = 'inner',
    left_index = True,
    right_on = 'Subject'
)
visit953Subj = visit953Img.groupby("Subject").aggregate({"DX": list, "Image Data ID": list})
visit953Subj = pd.merge(
    visit953Subj,
    visit953[['DX_vis2']],
    how = 'inner',
    left_index = True,
    right_index = True
)
visit953Subj

,DX,Image Data ID,DX_vis2
Subject,,,
002_S_0295,[CN],[I240519],CN-puro
002_S_0413,"[CN, CN]","[I241476, I384689]",CN-puro
002_S_0685,"[CN, CN]","[I187801, I323915]",CN-puro
002_S_0729,"[AD, AD]","[I189823, I324214]",AD-puro
002_S_1261,"[CN, CN]","[I221922, I363624]",CN-puro
...,...,...,...
941_S_4376,[CN],[I283673],CN-puro
941_S_5124,[CN],[I373385],CN-puro
941_S_5193,[CN],[I378347],CN-puro


Queremos obtener para AD la última imágen con AD (tanto puro como mixto, o podría ser una random), para el resto la primera imágen.

In [193]:
def processImg(x):
    DX_vis2 = x['DX_vis2']
    img = x['Image Data ID']
    
    if DX_vis2 in ['CN-puro']:
        # Devolvemos una imágen random si es CN puro
        return random.choice(img)
    
    if DX_vis2 in ['AD-puro']:
        # Devolvemos una imágen random si es AD puro
        return random.choice(img)
    
    # Si es no-AD, devolvemos el primero que sea CN
    if DX_vis2 == 'no-AD':
        selectedImg = None
        for i in range(0, len(img)):
            if x['DX'][i] == 'CN':
                selectedImg = img[i]
                break
        return selectedImg
                
    # Si es AD, devolvemos el primero de los últimos que sea AD
    if DX_vis2 == 'AD':
        selectedImg = None
        for i in reversed(range(0, len(img))):
            if x['DX'][i] == 'AD':
                selectedImg = img[i]
                break
        return selectedImg
    
    raise Exception(f"DX_vis2 desconocido: {DX_vis2}")
    
visit953Subj['Image Data ID'] = visit953Subj.apply(processImg, axis = 1)

In [194]:
visit953Subj

,DX,Image Data ID,DX_vis2
Subject,,,
002_S_0295,[CN],I240519,CN-puro
002_S_0413,"[CN, CN]",I384689,CN-puro
002_S_0685,"[CN, CN]",I187801,CN-puro
002_S_0729,"[AD, AD]",I324214,AD-puro
002_S_1261,"[CN, CN]",I221922,CN-puro
...,...,...,...
941_S_4376,[CN],I283673,CN-puro
941_S_5124,[CN],I373385,CN-puro
941_S_5193,[CN],I378347,CN-puro


Colapsamos los diagnósticos en la columna DX_vis

In [195]:
len(visit953Subj['Image Data ID'].unique())

953

In [196]:
def collapse_diag(x):
    diag = x['DX_vis2']
    if diag == 'CN-puro':
        return 'no-AD'
    if diag == 'AD-puro':
        return 'AD'
    return diag

visit953Subj['DX_vis'] = visit953Subj.apply(collapse_diag, axis = 1)
visit953Subj

,DX,Image Data ID,DX_vis2,DX_vis
Subject,,,,
002_S_0295,[CN],I240519,CN-puro,no-AD
002_S_0413,"[CN, CN]",I384689,CN-puro,no-AD
002_S_0685,"[CN, CN]",I187801,CN-puro,no-AD
002_S_0729,"[AD, AD]",I324214,AD-puro,AD
002_S_1261,"[CN, CN]",I221922,CN-puro,no-AD
...,...,...,...,...
941_S_4376,[CN],I283673,CN-puro,no-AD
941_S_5124,[CN],I373385,CN-puro,no-AD
941_S_5193,[CN],I378347,CN-puro,no-AD


Borramos la columna DX para que no contamine el PDF

In [197]:
visit953Subj = visit953Subj.drop(['DX'], axis = 1)

Ahora hacemos un merge para obtener las otras columnas y poder sacar la demografía

In [199]:
visit935Merge = pd.merge(
    visit953Subj,
    csvMergePerVisitFinal[['Image Data ID', 'Subject', 'Sex', 'Age', 'Visit', 'Acq Date', 'VISCODE2']],
    left_on = 'Image Data ID',
    right_on = 'Image Data ID',
    how = 'inner'
)
visit935Merge

,Image Data ID,DX_vis2,DX_vis,Subject,Sex,Age,Visit,Acq Date,VISCODE2
0,I240519,CN-puro,no-AD,002_S_0295,M,90,v06,2011-06-09,m60
1,I384689,CN-puro,no-AD,002_S_0413,F,83,v21,2013-05-14,m84
2,I187801,CN-puro,no-AD,002_S_0685,F,94,m48,2010-07-27,m48
3,I324214,AD-puro,AD,002_S_0729,F,71,v11,2012-08-14,m72
4,I221922,CN-puro,no-AD,002_S_1261,F,75,m48,2011-03-01,m48
...,...,...,...,...,...,...,...,...,...
948,I283673,CN-puro,no-AD,941_S_4376,F,77,v03,2012-02-08,bl
949,I373385,CN-puro,no-AD,941_S_5124,F,77,v03,2013-05-01,bl
950,I378347,CN-puro,no-AD,941_S_5193,F,73,v03,2013-06-19,bl
951,I1301874,AD-puro,AD,941_S_6854,M,86,bl,2020-02-24,bl


In [200]:
len(visit935Merge['Image Data ID'].unique())

953

In [201]:
visit935Merge.groupby(['Sex', 'DX_vis']).describe()

Age                                                   
            count       mean       std   min   25%   50%   75%   max
Sex DX_vis                                                          
F   AD      195.0  75.071795  8.068967  55.0  71.0  75.0  81.0  96.0
    no-AD   238.0  74.256303  6.364248  56.0  69.0  74.0  78.0  94.0
M   AD      291.0  76.962199  7.626764  56.0  72.5  78.0  82.0  92.0
    no-AD   229.0  75.480349  6.472330  62.0  71.0  75.0  80.0  90.0

In [202]:
visit935Merge.groupby(['DX_vis']).describe()

Age                                                   
        count       mean       std   min   25%   50%   75%   max
DX_vis                                                          
AD      486.0  76.203704  7.853928  55.0  72.0  77.0  82.0  96.0
no-AD   467.0  74.856531  6.439782  56.0  70.0  75.0  79.0  94.0

In [203]:
visit935Merge

,Image Data ID,DX_vis2,DX_vis,Subject,Sex,Age,Visit,Acq Date,VISCODE2
0,I240519,CN-puro,no-AD,002_S_0295,M,90,v06,2011-06-09,m60
1,I384689,CN-puro,no-AD,002_S_0413,F,83,v21,2013-05-14,m84
2,I187801,CN-puro,no-AD,002_S_0685,F,94,m48,2010-07-27,m48
3,I324214,AD-puro,AD,002_S_0729,F,71,v11,2012-08-14,m72
4,I221922,CN-puro,no-AD,002_S_1261,F,75,m48,2011-03-01,m48
...,...,...,...,...,...,...,...,...,...
948,I283673,CN-puro,no-AD,941_S_4376,F,77,v03,2012-02-08,bl
949,I373385,CN-puro,no-AD,941_S_5124,F,77,v03,2013-05-01,bl
950,I378347,CN-puro,no-AD,941_S_5193,F,73,v03,2013-06-19,bl
951,I1301874,AD-puro,AD,941_S_6854,M,86,bl,2020-02-24,bl


In [204]:
len(visit935Merge['Image Data ID'].unique())

953

In [205]:
duplicates = visit935Merge.duplicated()
print("Number of duplicates in visit935Merge:", duplicates.sum())

Number of duplicates in visit935Merge: 0


In [206]:
# len(visit935Merge['Subject'].unique())

In [207]:
print(randomState)

2234289712


Uso shuffling porque por algún motivo andaba mal al intentar usar sample (repetía rows):

In [208]:
# Shuffle the DataFrame
visit935Merge_shuffled = visit935Merge.sample(frac=1, random_state=randomState)

# Split into train, validation, and test sets
train_end = int(trainProportion * len(visit935Merge_shuffled))
val_end = train_end + int(valProportion * len(visit935Merge_shuffled))

visit935Train = visit935Merge_shuffled[:train_end]
visit935Val = visit935Merge_shuffled[train_end:val_end]
visit935Test = visit935Merge_shuffled[val_end:]

# Resetting the index for each DataFrame if needed
visit935Train.reset_index(drop=True, inplace=True)
visit935Val.reset_index(drop=True, inplace=True)
visit935Test.reset_index(drop=True, inplace=True)

Ahora dividimos entre train, validación y test

In [209]:
len(visit935Train)

762

In [210]:
len(visit935Val)

95

In [211]:
len(visit935Test)

96

Chequeamos que no haya overlapping de Image IDS

In [212]:
# Step 1: Get unique Image Data IDs from each DataFrame
unique_ids_train = visit935Train['Image Data ID'].unique()
unique_ids_val = visit935Val['Image Data ID'].unique()
unique_ids_test = visit935Test['Image Data ID'].unique()

# Step 2: Check for overlaps
overlap_train_val = set(unique_ids_train).intersection(unique_ids_val)
overlap_train_test = set(unique_ids_train).intersection(unique_ids_test)
overlap_val_test = set(unique_ids_val).intersection(unique_ids_test)

print("----")
print(len(overlap_train_val))
print("----")
print(len(overlap_train_test))
print("----")
print(len(overlap_val_test))

# Step 3: Print the results
print("Overlap between visit935Train and visit935Val:", bool(overlap_train_val))
print("Overlap between visit935Train and visit935Test:", bool(overlap_train_test))
print("Overlap between visit935Val and visit935Test:", bool(overlap_val_test))

----
0
----
0
----
0
Overlap between visit935Train and visit935Val: False
Overlap between visit935Train and visit935Test: False
Overlap between visit935Val and visit935Test: False


In [213]:
visit935Train

,Image Data ID,DX_vis2,DX_vis,Subject,Sex,Age,Visit,Acq Date,VISCODE2
0,I385859,no-AD,no-AD,029_S_5219,M,76,v03,2013-08-13,bl
1,I283673,CN-puro,no-AD,941_S_4376,F,77,v03,2012-02-08,bl
2,I334294,AD-puro,AD,036_S_4894,F,61,v03,2012-09-04,bl
3,I373714,AD-puro,AD,036_S_5112,F,75,v03,2013-05-13,bl
4,I89831,AD-puro,AD,130_S_1201,F,77,m12,2008-02-05,m12
...,...,...,...,...,...,...,...,...,...
757,I337234,AD-puro,AD,037_S_4879,F,80,v03,2012-09-26,bl
758,I943962,CN-puro,no-AD,168_S_6062,F,68,bl,2017-12-05,bl
759,I258863,AD-puro,AD,116_S_4209,F,78,v03,2011-09-23,bl
760,I375008,CN-puro,no-AD,098_S_4003,F,74,v21,2013-04-19,m24


In [214]:
visit935Train.groupby(['DX_vis']).describe()

Age                                                    
        count       mean       std   min    25%   50%   75%   max
DX_vis                                                           
AD      390.0  76.071795  7.930610  55.0  71.25  77.0  81.0  96.0
no-AD   372.0  74.758065  6.282699  56.0  70.00  75.0  79.0  93.0

In [215]:
visit935Val

,Image Data ID,DX_vis2,DX_vis,Subject,Sex,Age,Visit,Acq Date,VISCODE2
0,I383881,CN-puro,no-AD,127_S_5228,F,78,v03,2013-07-25,bl
1,I158142,AD,AD,100_S_0892,F,76,m36,2009-10-23,m36
2,I137371,CN-puro,no-AD,027_S_0120,M,75,m36,2009-02-27,m36
3,I27681,AD-puro,AD,035_S_0341,M,75,m06,2006-10-12,m06
4,I384714,CN-puro,no-AD,099_S_4076,F,75,v21,2013-07-30,m24
...,...,...,...,...,...,...,...,...,...
90,I281584,CN-puro,no-AD,032_S_4386,F,85,v03,2012-01-19,bl
91,I133138,AD,AD,006_S_0675,F,81,m24,2008-12-23,m24
92,I266957,CN-puro,no-AD,021_S_4254,F,86,v03,2011-11-08,bl
93,I356281,AD-puro,AD,130_S_4997,F,61,v03,2013-01-15,bl


In [216]:
visit935Val.groupby(['DX_vis']).describe()

Age                                                    
       count       mean       std   min   25%   50%    75%   max
DX_vis                                                          
AD      48.0  75.854167  7.348439  56.0  73.0  76.5  81.25  89.0
no-AD   47.0  75.127660  7.617534  64.0  69.0  73.0  81.50  90.0

In [217]:
visit935Test

,Image Data ID,DX_vis2,DX_vis,Subject,Sex,Age,Visit,Acq Date,VISCODE2
0,I225971,CN-puro,no-AD,100_S_1286,F,80,m48,2011-03-03,m48
1,I321658,CN-puro,no-AD,073_S_4762,M,74,v03,2012-07-24,bl
2,I77185,no-AD,no-AD,010_S_0422,M,63,m12,2007-09-27,m12
3,I128833,AD,AD,057_S_0941,F,76,m24,2008-11-18,m24
4,I325148,CN-puro,no-AD,127_S_4843,F,74,v03,2012-08-09,bl
...,...,...,...,...,...,...,...,...,...
91,I173995,AD,AD,057_S_1265,F,85,m36,2010-03-18,m36
92,I1248363,AD-puro,AD,168_S_6754,M,78,bl,2019-10-25,bl
93,I1146753,AD-puro,AD,135_S_6687,F,80,bl,2019-03-15,bl
94,I324435,CN-puro,no-AD,013_S_4579,M,85,v03,2012-06-12,bl


In [218]:
visit935Test.groupby(['DX_vis']).describe()

Age                                                     
       count       mean       std   min    25%   50%    75%   max
DX_vis                                                           
AD      48.0  77.625000  7.722928  57.0  72.00  78.0  84.25  90.0
no-AD   48.0  75.354167  6.511821  63.0  70.75  74.0  79.25  94.0

Guardamos todo

In [219]:
visit935Train.to_csv(dxVis953TrainOutputFile)
visit935Val.to_csv(dxVis953ValOutputFile)
visit935Test.to_csv(dxVis953TestOutputFile)

# Revisión DB ya creada

Esta revisión es para ver si el set de validación que estoy usando está bien o tiene alguna particularidad.

In [215]:
dfLast_train = pd.read_csv(dxLastTrainOutputFile)
dfLast_val = pd.read_csv(dxLastValOutputFile)
dfLast_test = pd.read_csv(dxLastTestOutputFile)

In [216]:
dfLast_val.head()

,Unnamed: 0,Image Data ID,Subject,Group,Sex,Age,Visit,Modality,Description,Type,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
0,0,I267498,002_S_4237,EMCI,F,81,v03,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,...,002_S_4237,"['MCI', 'MCI', 'MCI', 'MCI']","['bl', 'm06', 'm12', 'm24']",2011-10-13,2013-10-22,24.312614,MCI,MCI,0,"MCI,MCI,MCI,MCI"
1,1,I398953,002_S_4237,EMCI,F,83,v21,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,...,002_S_4237,"['MCI', 'MCI', 'MCI', 'MCI']","['bl', 'm06', 'm12', 'm24']",2011-10-13,2013-10-22,24.312614,MCI,MCI,0,"MCI,MCI,MCI,MCI"
2,2,I311756,002_S_4746,LMCI,F,71,v03,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,...,002_S_4746,"['MCI', 'AD', 'MCI', 'MCI', 'MCI', 'MCI']","['m48', 'm36', 'm24', 'm12', 'm06', 'bl']",2012-06-14,2016-06-23,48.296680,MCI,AD,1,"MCI,AD,MCI,MCI,MCI,MCI"
3,3,I376519,002_S_5178,SMC,M,69,v03,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,...,002_S_5178,"['CN', 'CN', 'CN', 'CN', 'CN']","['m24', 'm06', 'bl', 'm48', 'm78']",2013-05-28,2019-12-09,78.391753,CN,CN,0,"CN,CN,CN,CN,CN"
4,4,I334485,003_S_4892,AD,F,75,v03,PET,"Coreg, Avg, Std Img and Vox Siz, Uniform Resol...",Processed,...,003_S_4892,"['AD', 'AD', 'AD']","['m12', 'm06', 'bl']",2012-09-11,2013-09-19,12.254872,AD,AD,0,"AD,AD,AD"


In [217]:
len(dfLast_val)

357

In [218]:
len(dfLast_val['Subject'].unique())

164

In [219]:
len(dfLast_test)

359

In [220]:
len(dfLast_test['Subject'].unique())

166

El número de sujetos únicos es parecido. Pero y la distribución?

In [221]:
dfLast_val.groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,...,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000
mean,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829,...,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829,2.176829
std,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162,...,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162,1.813162
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


O sea tenemos una media de 2.17 y un std de 1.81.

In [222]:
dfLast_test.groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,...,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000,166.000000
mean,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651,...,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651,2.162651
std,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243,...,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243,1.752243
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


La media es parecida en test y el std también. 2.16 y 1.75.

In [223]:
dfLast_train.groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,...,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000
mean,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696,...,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696,2.256696
std,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073,...,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073,1.808073
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


Vemos que para train es muy similar... pero qué pasa con los grupos didividos por AD, etc.?

In [225]:
dfLast_val[dfLast_val["DX_last"] == "AD"].groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,...,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000,51.000000
mean,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588,...,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588,2.470588
std,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796,...,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796,1.792796
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000


In [226]:
dfLast_val[dfLast_val["DX_last"] == "MCI"].groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,...,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000,70.000000
mean,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429,...,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429,2.071429
std,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368,...,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368,1.812368
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


In [227]:
dfLast_val[dfLast_val["DX_last"] == "CN"].groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,...,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000,43.000000
mean,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
std,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737,...,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737,1.838737
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


Vemos que no se alejan del promedio 2 y 1.8 para std... pareciera estar bien entonces.

In [228]:
dfLast_train[dfLast_train["DX_last"] == "AD"].groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,...,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000,406.000000
mean,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113,...,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113,2.559113
std,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518,...,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518,2.002518
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [229]:
dfLast_train[dfLast_train["DX_last"] == "MCI"].groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000,...,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000,588.000000
mean,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136,...,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136,2.090136
std,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899,...,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899,1.701899
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000


In [230]:
dfLast_train[dfLast_train["DX_last"] == "CN"].groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,...,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000,350.000000
mean,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714,...,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714,2.185714
std,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953,...,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953,1.701953
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,...,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000


En train la media y el std sube un poco para AD pero nada grave...

In [231]:
dfLast_test[dfLast_test["DX_last"] == "AD"].groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,...,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000,48.000000
mean,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167,...,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167,2.354167
std,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498,...,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498,1.929498
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,...,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000
max,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [232]:
dfLast_test[dfLast_test["DX_last"] == "MCI"].groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,...,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526,...,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526,1.960526
std,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858,...,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858,1.668858
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,...,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


In [233]:
dfLast_test[dfLast_test["DX_last"] == "CN"].groupby("Subject").aggregate("count").describe()

,Unnamed: 0,Image Data ID,Group,Sex,Age,Visit,Modality,Description,Type,Acq Date,...,PTID,DX,VISCODE,EXAMDATE_min,EXAMDATE_max,EXAMLENGTH,DX_last,DX_most_severe,regressions,DX_hashed
count,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,...,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000
mean,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524,...,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524,2.309524
std,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106,...,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106,1.689106
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
50%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
75%,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,...,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000,7.000000


Para CN y MCI el std disminuye en test, aunque no mucho. El mean tambien es mas alto para test en CN y AD. Podríamos decir que el conjunto de test en realidad es peor que el de validación (por eso da mejor?).

In [234]:
dfLast_val.groupby("DX_last").describe()

Unnamed: 0                                                             \
             count        mean         std  min     25%    50%     75%    max   
DX_last                                                                         
AD           126.0  155.388889   95.135018  4.0   69.00  153.5  243.25  355.0   
CN            86.0  169.127907  113.977479  3.0   44.25  166.5  276.75  354.0   
MCI          145.0  202.910345   98.420012  0.0  119.00  205.0  299.00  356.0   

           Age             ... EXAMLENGTH             regressions            \
         count       mean  ...        75%         max       count      mean   
DX_last                    ...                                                
AD       126.0  74.896825  ...  49.052342  134.573605       126.0  0.087302   
CN        86.0  75.267442  ...  97.077969  180.570443        86.0  0.313953   
MCI      145.0  74.565517  ...  97.250457  176.332163       145.0  0.468966   

                                            
              std  min  25%  50%  75%  max  
DX_last                                     
AD       0.283403  0.0  0.0  0.0  0.0  1.0  
CN       0.618568  0.0  0.0  0.0  0.0  2.0  
MCI      0.635251  0.0  0.0  0.0  1.0  2.0  

[3 rows x 32 columns]

In [235]:
dfLast_test.groupby("DX_last").describe()

Unnamed: 0                                                            \
             count        mean         std   min    25%    50%    75%    max   
DX_last                                                                        
AD           113.0  171.327434   89.747408  10.0  103.0  161.0  257.0  349.0   
CN            97.0  170.371134  108.488797   3.0   75.0  154.0  237.0  358.0   
MCI          149.0  190.436242  109.984048   0.0   85.0  199.0  294.0  353.0   

           Age             ...  EXAMLENGTH             regressions            \
         count       mean  ...         75%         max       count      mean   
DX_last                    ...                                                 
AD       113.0  76.902655  ...   78.917432  164.208711       113.0  0.477876   
CN        97.0  73.350515  ...  134.540750  186.320048        97.0  0.340206   
MCI      149.0  75.302013  ...   84.009939  189.966940       149.0  0.255034   

                                            
              std  min  25%  50%  75%  max  
DX_last                                     
AD       1.102735  0.0  0.0  0.0  0.0  4.0  
CN       0.627273  0.0  0.0  0.0  1.0  2.0  
MCI      0.452536  0.0  0.0  0.0  0.0  2.0  

[3 rows x 32 columns]

En síntesis, el set de validación no parece tener ninguna particularidad. Faltaría revisar todas sus imágenes para estar seguro de que no hay nada raro. Los máximos de hecho revelan que el set de validación es el mejor de los 3.